# MemoryBanditWorkflow を使った RAG の試験実装

(Version: 0.0.2.1)

先に MemoryBanditWorkflow というフレームワークを作った。それは迷路問題の一部として作り始めたが、汎用なものになるようこころがけた。それが実際汎用に使えることを示すために、RAG エージェントを作ってみようというのが今回の試みである。

アイデアの軌跡は↓にある。

\[cocolog:95619779\](2025年9月)  
《「LLM のメモリ機能を強制的に使うバンディットマシンの試験実装」と「LLM のメモリ機能とバンディット機能の試験実装」を行った。後者がメインの成果物で、メモリ機能の使用増加をどう強制するかから拡張したフレームワーク。 - JRF のひとこと》  
http://jrf.cocolog-nifty.com/statuses/2025/09/post-8225e2.html

基本的にはメモリ機能とバンディット機能とワークフロー機能を持っているのが MemoryBanditWorkflow で、今回の物はそれをそのまま使って、RagAgent という子クラスを定義している。

なお、MemoryBanditWorkflow はセマンティックサーチとかを用意するのは、めんどくさかったので、そういうバックエンドはこれまた AI さんに「データベース偽装」をしてもらっている。

MemoryBanditWorkflow の元となった迷路問題は↓からたどっていただきたい。

《JRF-2018/langchain_maze: 熊剣迷路問題 revisited》  
https://github.com/JRF-2018/langchain_maze

ところで、RAG エージェントといえばマルチエージェントを使うのが流行りのようだが、今回の物はリニアに順次、シングルスレッドで実行されていくだけである。マルチエージェントを敷衍するなら asyncio を使ったり、メモリにデータベースを使ったりすべきなのだが、そういうことはできる展望すらない。あくまで MemoryBanditWorkflow で RAG も実装できることを示そうとしたものなので、RAG としての新規性はない。


## 前々回へのリンク

《experimental_rag_0.0.1.ipynb - GitHub Gist》  
https://gist.github.com/JRF-2018/6d7dda3eaf7743b91bca32037574bfc1

## 前々回からの変更点

MemoryBanditWorkflow に変更はない。

0.0.1 からは RAG のループを少し変えて、一章ごと取材・執筆をするように指示を出し、バンディットが呼ばれやすくならないか試してみた。

0.0.2 から 0.0.2.1 への変更は、実行制御の全体像を AI に伝えるための show_execution ツールの導入のみ。

## 結論

Gemini 2.5 Pro さんというコストのかかる遅いモデルを使った。やはり頭がいい者にやらせたかったから。前々回(0.0.1)は修正しながら load して resume をやった。前回(0.0.2)は、ほぼ一発出しで、タイトルの設定だけお忘れになった。今回は途中で終了するなど一発出しにはならなかったが、write_loop に入らせることでの二発出しで、論文としては前回より完成度が高いと思う。

一章ずつというわけにはいかなかったが、おおむね細かい単位で取材してくれて、バンディットが強制されやすくはなっていたと思う。前々回に引き続き MemoryBanditWorkflow の実証実験としてはそこそこうまくいったと思う。

なお、再現性がないため、最初からの再実行はやっていないのはご容赦願いたい。コストもかかるしね \(^^;。


## 著者

JRF ( http://jrf.cocolog-nifty.com/statuses , Twitter (X): @jion_rockford )


## ライセンス

基本短いコードなので(私が作った部分は)パブリックドメインのつもりです。気になる方は MIT License として扱ってください。

かなり AI さん達(Gemini さんや ChatGPT さんや Claude さんや Grok さん)に教わって作っています。

## 実装

まず、必要なライブラリを読み込みます。

In [1]:
!pip install -q -U langchain langgraph langchain-google-genai langmem duckduckgo-search langchain-community beautifulsoup4 ddgs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.7 MB/

Gemini にアクセスします。シークレットで Gemini API キーを Google AI Studio からインポートすると GOOGLE_API_KEY というシークレットができるはずです。それを使います。

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from google.colab import userdata

llm = ChatGoogleGenerativeAI(google_api_key=userdata.get('GOOGLE_API_KEY'), model="models/gemini-2.5-pro")
emb_llm = GoogleGenerativeAIEmbeddings(google_api_key=userdata.get('GOOGLE_API_KEY'), model='gemini-embedding-001')

ちゃんと Gemini にアクセスできるかテストします。

In [3]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(template="以下の質問に回答してください：{question}", input_variables=["question"])
chain = LLMChain(llm=llm, prompt=prompt)
response = chain.run({"question": "Geminiモデルの特徴を教えてください"})
print(response)

/tmp/ipython-input-1709569937.py:5: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)
/tmp/ipython-input-1709569937.py:6: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run({"question": "Geminiモデルの特徴を教えてください"})


はい、承知いたしました。Googleが開発したAIモデル「Gemini」の主な特徴について、分かりやすくご説明します。

Geminiは、これまでのAIモデルを大きく超える性能を持つ、非常に強力で柔軟なAIモデルです。その最大の特徴は以下の点に集約されます。

### 1. ネイティブなマルチモーダリティ（Multimodality）
これがGeminiの最も革新的な特徴です。従来のAIはテキストならテキスト、画像なら画像と、別々に学習していましたが、Geminiは**最初からテキスト、画像、音声、動画、コードなど、様々な種類の情報を同時に理解し、処理できるように設計**されています。

*   **具体例：**
    *   **画像とテキストの融合:** ホワイトボードに描かれた物理の問題の図と数式をスマートフォンで撮影し、「この問題を解いて、解き方をステップごとに説明して」と質問すると、Geminiは図と文字の両方を理解して解答を導き出します。
    *   **動画の理解:** 料理動画を見せて、「このレシピの材料リストを作って」と指示すると、動画の内容を理解してテキストでリストを作成します。
    *   **音声と画像の連携:** 話し声を聞きながら、その内容に合った画像をリアルタイムで生成することも可能です。

このように、複数の情報を人間のように統合的に捉え、高度な推論を行うことができます。

### 2. 非常に高度な推論能力
Geminiは、複雑な情報を理解し、論理的に考えて答えを導き出す能力が飛躍的に向上しています。

*   **専門分野での活用:** 数学、物理、プログラミングといった専門的な分野で、大学レベルの難問を解くことができます。
*   **多段階の思考:** 複数のステップを踏まなければ解けないような複雑な問題に対しても、計画を立てて順序だてて解決策を導き出すことができます。

### 3. 用途に応じた柔軟なモデルサイズ
Geminiは、性能と用途に応じて3つのサイズが用意されています。これにより、大規模なデータセンターから個人のスマートフォンまで、様々な環境で最適に動作します。

*   **Gemini Ultra:** 最も高性能で巨大なモデル。非常に複雑で大規模なタスク向け。
*   **Gemini Pro:*

埋め込みベクトルも試しておきます。

In [4]:
emb_llm.embed_query("これはテストです。")[:5]

[-0.013715926557779312,
 -0.0006048799259588122,
 0.011018728837370872,
 -0.09579180181026459,
 0.006502930540591478]

基本的なモジュールを読み込みます。

In [5]:
import os
import math
import numpy as np
import random
import re
from pprint import pprint
from time import sleep
import pickle
np.set_printoptions(legacy='1.25')

セーブ／ロードのために次のコードを実行します。

In [6]:
RAG_AGENT_SAVE = "rag-agent.pickle"

ライブラリ集。

In [7]:
from typing import List, Dict, Any, Tuple, Union
from textwrap import dedent
import datetime
import copy
from IPython.display import Markdown

# LangGraphのコンポーネントをインポート
from langchain_core.tools import tool, Tool
from langgraph.prebuilt import create_react_agent
#from langchain_core.messages.utils import count_tokens_approximately
from langgraph.prebuilt.chat_agent_executor import AgentState
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.prompts.chat import ChatPromptTemplate
from langmem.short_term import SummarizationNode, summarize_messages
from langchain_core.messages import AIMessage, ToolMessage, HumanMessage, SystemMessage
from langgraph.errors import GraphRecursionError
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_community.document_loaders import WebBaseLoader

INITIAL_SUMMARY_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("placeholder", "{messages}"),
        ("user", "上記の会話の要約を作成してください:"),
    ]
)

EXISTING_SUMMARY_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("placeholder", "{messages}"),
        (
            "user",
            "これまでの会話の要約です: {existing_summary}\n\n"
            "上記の新しいメッセージを考慮して、この要約を拡張してください:",
        ),
    ]
)

FINAL_SUMMARY_PROMPT = ChatPromptTemplate.from_messages(
    [
        # if exists
        ("placeholder", "{system_message}"),
        ("system", "これまでの会話の要約: {summary}"),
        ("placeholder", "{messages}"),
    ]
)


In [8]:
def calc_embedding_variance(embeddings):
    if not embeddings or len(embeddings) < 2:
        return 0.0

    embeddings_array = np.array(embeddings)
    mean_vector = np.mean(embeddings_array, axis=0)
    squared_distances = np.linalg.norm(embeddings_array - mean_vector, axis=1)**2
    variance = np.mean(squared_distances)

    return variance


def short_repr(x, max_len=80):
    repr_str = repr(x)

    if len(repr_str) > max_len:
        ellipsis_len = 3

        head_len = max_len - ellipsis_len - 1
        tail_len = 1
        return repr_str[:head_len] + "..." + repr_str[-tail_len:]
    else:
        return repr_str


MemoryBanditWorkflow は変わっていないです。長大な定義で申し訳ないです。

In [9]:
class MemoryBanditWorkflow:
    def __init__ (self, llm=llm, llm2=llm, emb_llm=emb_llm,
                  save_file=None):
        self.llm = llm
        self.llm2 = llm2
        self.emb_llm = emb_llm
        self.save_file = save_file

        self.plan = "まだ計画と方針はセットされていません。"
        self.scratchpad = ""

        self.messages = []
        self.running_summary = None
        self.system_prompt = """\
これはメモリ機能とバンディット機能の動きを見るための実験です。
現在の計画と方針と周囲の状況を考慮し、必要に応じて計画と方針を更新してください。
あなたとは別の者が次の行動をしやすいよう計画と方針を残してください。
実験にふさわしく、できる限り、ツールを利用し、特に、メモリを検索し、その文書を更新して、後のコマンドに備えてください。
メモリの id は memory:～ という形式です。memory:5555 を指定するときは 5555 だけではダメです。文の中でメモリの参照を示すときは [memory:〜] のように書きます。
「メモリの文書を検索する手順」は [memory:9998] にあります。「メモリに文書を残す手順」は [memory:9997] にあります。
"""

        self.backend_status = None
        self.backend_result = None
        self.messages2 = []
        self.system_prompt2 = """\
エージェントをサポートする裏方のエージェントです。
本来この裏方は様々な技法を用いて実装される予定なのですが、現在は試験実装中のため、その動きをあなたは偽装しなければなりません。

よく考えツールを積極的に使って Human からの指示に従ってください。
"""

        self.memories = {}
        self.keywords = []

        self.access_unit = 1.0
        self.recent_reads = []

        self.workflows = {}
        self.workflow_current = "workflow:main"
        self.workflow_next = None
        self.privileged_tool_names = []

        self.init_memories()
        self.init_workflows()


    def __getstate__ (self):
        state = self.__dict__.copy()
        del state['llm']
        del state['llm2']
        del state['emb_llm']
        #del state['agent']
        return state

    def __setstate__ (self, state):
        self.__dict__.update(state)
        self.prev_load = True

    def save (self):
        if not self.save_file:
            return
        with open(self.save_file, 'wb') as f:
            pickle.dump(self, f)

    @classmethod
    def load (cls, filename, llm=llm, llm2=llm, emb_llm=emb_llm):
        with open(filename, 'rb') as f:
            loaded_game = pickle.load(f)
        loaded_game.llm = llm
        loaded_game.llm2 = llm2
        loaded_game.emb_llm = emb_llm
        return loaded_game

    def normalize_memory_id(self, id_or_num):
        if isinstance(id_or_num, int):
            return f"memory:{id_or_num}"
        elif isinstance(id_or_num, str):
            m = re.search(r'\[?memory:(\d+)\]?', id_or_num)
            if m:
                return f"memory:{m.group(1)}"
            if id_or_num.isdigit():
                return f"memory:{id_or_num}"
            else:
                return id_or_num
        else:
            return id_or_num

    def _normalize_workflow_id_sub(self, id_or_num):
        if isinstance(id_or_num, int):
            return f"workflow:{id_or_num}"
        if id_or_num in ["current", "main"]:
            return f"workflow:{id_or_num}"
        elif isinstance(id_or_num, str):
            m = re.search(r'\[?workflow:(\d+|main|current)\]?(?:.+)?', id_or_num.strip())
            if m:
                return f"workflow:{m.group(1)}"
            if id_or_num.isdigit():
                return f"workflow:{id_or_num}"
            else:
                return id_or_num
        else:
            return id_or_num

    def normalize_workflow_id(self, id_or_num):
        r = self._normalize_workflow_id_sub(id_or_num)
        if r == "workflow:current":
            return self.workflow_current
        return r

    def _replace_tools (self, from_tools, to_tools):
        tool_names = [x.name for x in to_tools]
        return [x for x in from_tools
                if x.name not in tool_names] + to_tools

    def _create_tools (self):
        @tool
        def express_thought (thought: str) -> None:
            """
            プレイヤーの現在の考えを吐露します。
            """
            mes = f"「{thought}」と考えが吐露されました。"
            print(f"ツール(express_thought): {mes}")

        @tool
        def update_plan (new_plan: str) -> str:
            """
            プレイヤーの現在の計画と方針を更新します。
            表示されるべき新しい計画と方針の文字列を提供してください。
            あなたとは別の者が次の行動をしやすいよう計画と方針を残してください。
            """
            self.plan = new_plan
            mes = "計画と方針が更新されました。"
            print(f"ツール(update_plan): {mes}: {new_plan}")
            return mes

        @tool
        def show_plan () -> str:
            """
            プレイヤーの現在の計画と方針を返します。
            """
            print(f"ツール(show_plan): {self.plan}")
            return self.plan

        @tool
        def update_scratchpad (new_scratchpad: str) -> str:
            """
            自由に使えるスクラッチパッドを更新します。
            """
            self.scratchpad = new_scratchpad
            mes = "スクラッチパッドが更新されました。"
            print(f"ツール(update_scratchpad): {mes}: {new_scratchpad}")
            return mes

        @tool
        def show_scratchpad () -> str:
            """
            スクラッチパッドを返します。
            """
            print(f"ツール(show_scratchpad): {self.scratchpad}")
            return self.scratchpad

        @tool
        def memory_new (title: str, text: str) -> str:
            """
            指定された title と text によるメモリを構成します。

            Returns:
                str: 割り当てられた memory_id。
            """

            i = 1000
            while True:

                if f"memory:{i}" not in self.memories:
                    break
                i = i + 1
            new_id = f"memory:{i}"
            self.memories[new_id] = {
                'id': new_id,
                'title': title,
                'accesses': 0,
                'text': text,
                'modified_at': datetime.datetime.now().isoformat()
            }
            self.update_keywords(text)
            self.update_vector(self.memories[new_id])
            print(f"ツール(memory_new): {short_repr(self.memories[new_id])}")
            return new_id

        @tool
        def memory_update_string (
                memory_id: str,
                from_str: str,
                to_str: str
        ) -> str:
            """
            指定されたmemory_idの記憶の中にある文字列を修正します。

            Args:
                memory_id (str): 修正する記憶のID。
                from_str (str): 置換元の文字列を含む文字列。
                to_str (str): 置換先の文字列を含む文字列。

            Returns:
                str: 処理結果を説明する簡潔なメッセージ。
            """

            memory_id = self.normalize_memory_id(memory_id)
            if memory_id not in self.memories:
                return f"エラー: 記憶ID '{memory_id}' が見つかりませんでした。"

            if memory_id.startswith("memory:9"):
                return f"エラー:  [{memory_id}] の書き換えは禁じられています。"

            original_title = self.memories[memory_id]['title']
            original_text = self.memories[memory_id]['text']

            if from_str not in original_text and from_str not in original_title:
                return f"エラー: 置換元の文字列 '{from_str}' が記憶内に見つかりませんでした。"

            updated_title = original_title.replace(from_str, to_str)
            updated_text = original_text.replace(from_str, to_str)

            self.memories[memory_id]['title'] = updated_title
            self.memories[memory_id]['text'] = updated_text
            self.memories[memory_id]['modified_at'] = datetime.datetime.now().isoformat()
            self.update_keywords(updated_text)
            self.update_vector(self.memories[memory_id])

            return f"成功: 記憶ID '{memory_id}' の文字列を '{from_str}' から '{to_str}' に修正しました。"

        @tool
        def memory_append_string (
                memory_id: str,
                string_to_append: str,
                separator: str = '\n'
        ) -> str:
            """
            指定されたmemory_idの記憶に文字列を追記します。
            """
            memory_id = self.normalize_memory_id(memory_id)
            if memory_id not in self.memories:
                return f"エラー: 記憶ID '{memory_id}' が見つかりませんでした。"

            if memory_id.startswith("memory:9"):
                return f"エラー:  [{memory_id}] の書き換えは禁じられています。"


            original_text = self.memories[memory_id]['text']
            updated_text = original_text + separator + string_to_append
            self.memories[memory_id]['text'] = updated_text
            self.memories[memory_id]['modified_at'] = datetime.datetime.now().isoformat()
            self.update_keywords(updated_text)
            self.update_vector(self.memories[memory_id])

            return f"成功: 記憶ID '{memory_id}' に文字列 '{string_to_append}' を追記しました。"

        @tool
        def memory_delete (memory_id: str) -> str:
            """
            指定されたmemory_idの記憶に文字列を追記します。
            """
            memory_id = self.normalize_memory_id(memory_id)
            if memory_id not in self.memories:
                return f"エラー: 記憶ID '{memory_id}' が見つかりませんでした。"

            if memory_id.startswith("memory:9"):
                return f"エラー:  [{memory_id}] の書き換えは禁じられています。"

            del self.memories[memory_id]

            return f"成功: 記憶ID '{memory_id}' を削除しました。"

        @tool
        def memory_read(memory_id: str) -> Union[Dict[str, str], str]:
            """
            指定されたIDの記憶を読み込みます。

            Args:
                memory_id (str): 読み込む記憶のID。

            Returns:
                Union[Dict[str, str], str]: 記憶が見つかった場合は辞書、
                      見つからない場合はエラーメッセージ文字列を返します。
            """

            memory_id = self.normalize_memory_id(memory_id)
            if memory_id in self.memories:
                self.memories[memory_id]['accesses'] += self.access_unit * 1.0
                self.recent_reads.append(self.memories[memory_id])
                self.recent_reads = self.recent_reads[-10:]
                r = self.memories[memory_id].copy()
                del r['vector']
                return r
            else:
                return f"エラー: 記憶ID '{memory_id}' が見つかりませんでした。"

        @tool
        def memory_list_recent(top_n: int = 10) -> Dict[str, Any]:
            """
            最近変更されたメモリを新しい順でリストします。
            """

            filter_date = datetime.datetime(2025, 1, 1)
            sorted_memories = sorted(
                [memory for memory in self.memories.values()
                 if datetime.datetime.fromisoformat(memory['modified_at'])
                 >= filter_date],
                key=lambda x: datetime.datetime.fromisoformat(x['modified_at']),
                reverse=True
            )
            if sorted_memories:
                if len(sorted_memories) > top_n:
                    sorted_memories = sorted_memories[:top_n]
                r = [{'id': x['id'], 'title': x['title'],
                      'modified_at': x['modified_at']}
                     for x in sorted_memories]
                return {'status': 'success', 'result': r}
            else:
                return {'status': 'error',
                        'result': 'エラー: 最近のメモリはありません。'}

        @tool
        def memory_list_random(top_n: int = 10) -> Dict[str, Any]:
            """
            メモリをランダムにリストします。
            """

            keys = list(self.memories.keys())
            if len(keys) > top_n:
                keys = random.sample(keys, top_n)
            if keys:
                values = [self.memories[k] for k in keys]
                r = [{'id': x['id'], 'title': x['title'],
                      'modified_at': x['modified_at']}
                     for x in values]
                return {'status': 'success', 'result': r}
            else:
                return {'status': 'error',
                        'result': 'エラー: メモリはありません。'}

        @tool
        def memory_words_search(search_str: str) -> Dict[str, Any]:
            """
            メモリ内を search_str で文字列検索します。OR や () が使えます。
            """

            res = self.call_backend_agent(dedent(f"""\
            メモリ全部を search_str = {repr(search_str)} で文字列検索するのを偽装してください。OR や () が使えることになっています。

            ただし、メモリやキーワードは read_all_memories や read_all_keywords ツールで得られる本物のメモリやキーワードを使ってください。

            set_result ツールで結果を返してください。

            status は 'error' か 'success'。
            result は「マッチデータ」のリスト。
            マッチデータはそれぞれが辞書型、それを m とすると。
            m['id'] はメモリ id。memory:〜 という形式。
            m['title'] はメモリの title。
            m['snippet'] はメモリの text のその文字列にマッチする部分周辺。
            """))
            if res['status'] == 'success':
                for m in res['result']:
                    if 'id' in m and m['id'] in self.memories:
                        x = self.memories[m['id']]
                        x['accesses'] += self.access_unit * 0.1
            return res

        @tool
        def memory_semantic_search(search_str: str) -> Dict[str, Any]:
            """
            メモリ内を search_str でセマンティックサーチします。
            """

            res = self.call_backend_agent(dedent(f"""\
            メモリ全部を search_str = {repr(search_str)} でセマンティックにサーチするのを偽装してください。

            ただし、メモリは read_all_memories や ツールで得られる本物のメモリを使ってください。

            set_result ツールで結果を返してください。

            status は 'error' か 'success'。
            result は「マッチデータ」のリスト。
            マッチデータはそれぞれが辞書型、それを m とすると。
            m['id'] はメモリ id。memory:〜 という形式。
            m['title'] はメモリの title。
            m['snippet'] はメモリの text のそのセマンティックにマッチする部分周辺。
            """))
            if res['status'] == 'success':
                for m in res['result']:
                    if 'id' in m and m['id'] in self.memories:
                        x = self.memories[m['id']]
                        x['accesses'] += self.access_unit * 0.1
            return res

        @tool
        def imagine_keywords(thought: str) -> List[Tuple[str, float]]:
            """
            thought からキーワードをスコア付きで連想します。
            """

            r = self.call_backend_agent(dedent(f"""\
            thought = {repr(thought)} からキーワードをスコア付きで複数連想してください。

            ただし、キーワードは read_all_memories や read_all_keywords ツールで得られる本物のキーワードを使ってください。

            set_result ツールで結果を返してください。

            status は 'error' か 'success'。
            result は「キーワードデータ」のリスト。
            キーワードデータは文字列とスコアからなるタプル。
            """))
            if r['status'] == 'success':
                return r["result"]
            else:
                return []

        @tool
        def bandit_schedule(tool_name: str, times: int, prob: float, exec_mode: str = "persistent", aux_prompt: str = "", workflow_id: str = "workflow:current") -> str:
            """
            ツールの利用を強制するためのバンディットを予約します。

            Args:
                tool_name: 強制するツールの名前。" OR " で区切って複数登録できる。
                times: 一度で何回同じ物を登録するか。times == 0 にもできます。
                prob: 一回の実行がある確率。prob == 0.0 にもできます。
                exec_mode: "once" or "persistent"
                aux_prompt: 実行の詳細を示すための補助プロンプト または ""
                workflow_id: バンディットを予約するワークフローを指定します。

            Returns:
                str: "成功" または "失敗" とその理由を返します。
            """

            tool_names = re.split(r"\s+or\s+|\s+OR\s+", tool_name)
            prohibited = set(self.privileged_tool_names) & set(tool_names)
            if prohibited:
                return f"失敗。{repr(prohibited)} は登録できません。"
            all_tools = [x.name
                         for x in self._create_tools()['all_tools']]
            if not any (x in all_tools for x in tool_names):
                return f"失敗。{tool_name} は有効なツールでありません。"

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} は有効なワークフローではありません。"
            if 'w' in self.workflows[workflow_id]['pin']:
                return f"失敗。{workflow_id} は書き換え不能のワークフローです。"

            dest = None
            for i, x in enumerate(self.workflows[workflow_id]['stack']):
                if x['tool_name'] == tool_name \
                   and x['exec_mode'] == exec_mode \
                   and x['aux_prompt'] == aux_prompt \
                   and x['arg'] is None:
                    dest = i
                    break
            if dest is not None:
                x = self.workflows[workflow_id]['stack'][dest]
                if not x['pin']:
                    self.workflows[workflow_id]['stack'].pop(dest)
                    if times == 0 or prob == 0.0:
                        return "成功。バンディトを削除しました。"
                    self.workflows[workflow_id]['stack'].append(x)
            else:
                if times == 0 or prob == 0.0:
                    return "失敗。そのようなバンディットはありません。バンディットを指定するには tool_name exec_mode aux_prompt のすべてを一致させて指定する必要があります。"
                x = {
                    'pin': 'stack' if exec_mode != "once" else None,
                    'arg': None
                }
                self.workflows[workflow_id]['stack'].append(x)
            if x['pin'] == "write":
                return f"失敗。'{tool_name}' は書き換えられません。"
            else:
                x['tool_name'] = tool_name
                x['tools_name'] = 'default_tools'
                x['exec_mode'] = exec_mode
                x['aux_prompt'] = aux_prompt
                x['prob'] = prob
                x['times'] = times
                print(f"ツール(bandit_schedule): {repr(x)}")
                if dest is None:
                    return "成功。バンディットを登録しました。"
                else:
                    return "成功。バンディットを更新しました。"

        @tool
        def bandit_schedule_read(memory_id: str, times: int, prob: float, exec_mode: str = "persistent", workflow_id: str = "workflow:current") -> str:
            """
            memory_read ツールの利用を強制するためのバンディットを予約します。

            Args:
                memory_id: memory_read するための memory_id。
                times: 一度で何回同じ物を登録するか。times == 0 にもできます。
                prob: 一回の実行がある確率。prob == 0.0 にもできます。
                exec_mode: "once" or "persistent"
                workflow_id: バンディットを予約するワークフローを指定します。

            Returns:
                str: "成功" または "失敗" とその理由を返します。
            """

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} は有効なワークフローではありません。"
            if 'w' in self.workflows[workflow_id]['pin']:
                return f"失敗。{workflow_id} は書き換え不能のワークフローです。"

            memory_id = self.normalize_memory_id(memory_id)

            dest = None
            for i, x in enumerate(self.workflows[workflow_id]['stack']):
                if x['tool_name'] == "memory_read" \
                   and x['exec_mode'] == exec_mode \
                   and not x['aux_prompt'] \
                   and x['arg'] == memory_id:
                    dest = i
                    break
            if dest is not None:
                x = self.workflows[workflow_id]['stack'][dest]
                if not x['pin']:
                    self.workflows[workflow_id]['stack'].pop(dest)
                    if times == 0 or prob == 0.0:
                        return "成功。バンディトを削除しました。"
                    self.workflows[workflow_id]['stack'].append(x)
            else:
                if times == 0 or prob == 0.0:
                    return "失敗。そのようなバンディットはありません。バンディットを指定するには exec_mode memory_id のすべてを一致させて指定する必要があります。"
                x = {'pin': None, 'arg': memory_id}
                self.workflows[workflow_id]['stack'].append(x)
            if x['pin'] == "write":
                return f"失敗。'memory_read {memory_id}' は書き換えられません。"
            else:
                x['tool_name'] = 'memory_read'
                x['tools_name'] = 'read_tools'
                x['exec_mode'] = exec_mode
                x['aux_prompt'] = ""
                x['prob'] = prob
                x['times'] = times
                print(f"ツール(bandit_schedule_read): {repr(x)}")
                if dest is None:
                    return "成功。バンディットを登録しました。"
                else:
                    return "成功。バンディットを更新しました。"

        @tool
        def bandit_schedule_workflow(workflow_id_to_schedule: str, times: int, prob: float, exec_mode: str = "persistent", workflow_id: str = "workflow:current") -> str:
            """
            workflow_do ツールの利用を強制するためのバンディットを予約します。

            Args:
                workflow_id_to_schedule: workflow_do するための workflow_id。
                times: 一度で何回同じ物を登録するか。times == 0 にもできます。
                prob: 一回の実行がある確率。prob == 0.0 にもできます。
                exec_mode: "once" or "persistent"
                workflow_id: バンディットを予約するワークフローを指定します。

            Returns:
                str: "成功" または "失敗" とその理由を返します。
            """

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} は有効なワークフローではありません。"
            if 'w' in self.workflows[workflow_id]['pin']:
                return f"失敗。{workflow_id} は書き換え不能のワークフローです。"

            workflow_id2 = self.normalize_workflow_id(workflow_id_to_schedule)
            if workflow_id2 not in self.workflows:
                return f"失敗。{workflow_id2} は有効なワークフローではありません。"

            dest = None
            for i, x in enumerate(self.workflows[workflow_id]['stack']):
                if x['tool_name'] == "workflow_do" \
                   and x['exec_mode'] == exec_mode \
                   and not x['aux_prompt'] \
                   and x['arg'] == workflow_id2:
                    dest = i
                    break
            if dest is not None:
                x = self.workflows[workflow_id]['stack'][dest]
                if not x['pin']:
                    self.workflows[workflow_id]['stack'].pop(dest)
                    if times == 0 or prob == 0.0:
                        return "成功。バンディトを削除しました。"
                    self.workflows[workflow_id]['stack'].append(x)
            else:
                if times == 0 or prob == 0.0:
                    return "失敗。そのようなバンディットはありません。バンディットを指定するには exec_mode workflow_id_to_schedule のすべてを一致させて指定する必要があります。"
                x = {
                    'pin': 'stack' if exec_mode != "once" else None,
                    'arg': workflow_id2
                }
                self.workflows[workflow_id]['stack'].append(x)
            if x['pin'] == "write":
                return f"失敗。'workflow_do {workflow_id2}' は書き換えられません。"
            else:
                x['tool_name'] = 'workflow_do'
                x['tools_name'] = 'default_tools'
                x['exec_mode'] = exec_mode
                x['aux_prompt'] = ""
                x['prob'] = prob
                x['times'] = times
                print(f"ツール(bandit_schedule_workflow): {repr(x)}")
                if dest is None:
                    return "成功。バンディットを登録しました。"
                else:
                    return "成功。バンディットを更新しました。"

        @tool
        def bandit_list(workflow_id: str =  "workflow:current")  -> Dict[str, Any]:
            """
            登録されているバンディットのスタックを返します。

            Args:
                workflow_id: バンディットのスタックが登録されたワークフローを指定します。
            """

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} は有効なワークフローではありません。"
            return {'status': 'success',
                    'result': self.workflows[workflow_id]['stack']}

        @tool
        def bandit_statistics()  -> str:
            """
            バンディットに有用かもしれない統計情報を返します。
            """
            s_read = calc_embedding_variance([
                x['vector'] for x in self.recent_reads
            ])
            s_write = calc_embedding_variance([
                x['vector'] for x in self.memories.values()
            ])
            accesses = [x['accesses'] for x in self.memories.values()]
            accesses.sort()
            accesses = accesses[:len(accesses) // 2]
            if accesses:
                s_access = np.mean(accesses)
            else:
                s_access = 0.0

            return dedent(f"""\
            最近 10個のメモリ read の分散: {s_read}
            メモリの分散: {s_write}
            下位50%のアクセス数の平均: {s_access}
            """)

        @tool
        def subwork_done()  -> str:
            """
            指示されたサブワークが完了したことを宣言します。サブワークが完了したときのみ使ってください。
            """

            return f"成功。サブワークの完了が宣言されました。"

        @tool
        def workflow_do(workflow_id: str)  -> str:
            """
            ワークフローを実行します。
            """

            self.workflow_next = None
            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} は有効なワークフローではありません。"
            if 'e' in self.workflows[workflow_id]['pin']:
                return f"失敗。{workflow_id} は子として実行できません。"
            self.workflow_next = workflow_id
            title = self.workflows[workflow_id]['title']
            return f"成功。この後、{workflow_id} 「{title}」を実行していきます。"

        @tool
        def workflow_list()  -> Dict[str, Any]:
            """
            登録されているすべてのワークフローの id とその title および pin 状況を返します。
            """

            return {'status': 'success',
                    'result': list(self.workflows.values())}

        @tool
        def workflow_show_current()  -> str:
            """
            現在実行中のワークフローのタイトル、および、強制しているバンディットの情報を表示します。
            """

            wtitle = self.workflows[self.workflow_current]['title']
            mes = dedent(f"""\
            現在のワークフロー: {self.workflow_current} 「{wtitle}」
            現在のバンディットの指示: 「{self.cur_bandit_prompt}」
            現在のバンディット: {repr(self.cur_bandit)}
            現在のバンディットの done 数: {self.cur_bandit_done}
            """)
            print(f"ツール(workflow_show_current): {mes}")
            return mes


        @tool
        def workflow_new(title: str, bandits: List[Dict[str, Any]], pin: str)  -> str:
            """
            新しいワークフローを定義し、その workflow_id を返します。

            Args:
                title: ワークフローの名前。
                bandits: 登録するバンディットのリスト。
                pin: 'w' は書換不能。'd' は削除不能。'wd' は書換＆削除不能。'' は書換＆削除可能。

            バンディットはそれぞれ辞書型、それを b とすると。
            b['tool_name'] は bandit_schedule での tool_name と同じ。
            b['exec_mode'] は bandit_schedule の exec_mode と同じ。
            b['aux_prompt'] は bandit_schedule の aux_prompt と同じ。
            b['prob'] は bandit_schedule の prob と同じ。
            b['times'] は bandit_schedule の times と同じ。
            b['arg'] は b['tool_name'] が 'memory_read' のとき memory:〜 という memory_id を指定できる。または、b['tool_name'] が 'workflow_do' のとき workflow:〜 という workflow_id を指定できる。
            b['pin'] は、None のとき削除または prob と times の書き換え可能。'stack' のときは prob と times の書き換え可能。'write' のときは書き換えも不能。
            """
            # AI に対する隠し属性として pin の 'e' は子として実行不能。

            for b in bandits:
                if not all(x in b for x in ['tool_name', 'exec_mode',
                                            'aux_prompt', 'times', 'prob']):
                    return f"失敗。有効な定義ではありません。"
                if 'arg' not in b:
                    b['arg'] = None
                if b['tool_name'] not in ["memory_read", "workflow_do"] \
                   and b['arg']:
                    return f"失敗。有効な定義ではありません。"
                if b['arg'] and b['tool_name'] == "memory_read":
                    b['tools_name'] = "read_tools"
                else:
                    b['tools_name'] = "default_tools"
                if 'pin' not in b:
                    b['pin'] = None
                if not (b['pin'] is None or b['pin'] == 'stack'
                        or b['pin'] == 'write'):
                    return f"失敗。不正な pin 値です。"
                tool_names = re.split(r"\s+or\s+|\s+OR\s+", b['tool_name'])
                prohibited = set(self.privileged_tool_names) & set(tool_names)
                if prohibited:
                    return f"失敗。{repr(prohibited)} は登録できません。"
                all_tools = [x.name
                             for x in self._create_tools()[b['tools_name']]]
                if not any (x in all_tools for x in tool_names):
                    return f"失敗。{b['tool_name']} は有効なツールでありません。"

            i = 1000
            while True:

                if f"workflow:{i}" not in self.workflows:
                    break
                i = i + 1
            new_id = f"workflow:{i}"

            self.workflows[new_id] = {'stack': bandits, 'pin': pin,
                                      'title': title}
            print(f"ツール(workflow_new): {repr(self.workflows[new_id])}")
            return f"成功。{new_id} を新規登録しました。"

        @tool
        def workflow_delete(workflow_id: str)  -> str:
            """
            ワークフローを削除します。
            """

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} はありません。"
            if 'd' in self.workflows[workflow_id]['pin']:
                return f"失敗。{workflow_id} は削除できません。"
            del self.workflows[workflow_id]
            return f"成功。{workflow_id} を削除しました。"

        r = {}
        r['null_tools'] = []
        r['read_tools'] = r['null_tools'] + [
            express_thought,
            update_plan, show_plan, update_scratchpad, show_scratchpad,
            memory_read, memory_list_recent, memory_list_random,
            memory_semantic_search, memory_words_search,
            imagine_keywords,
            bandit_schedule, bandit_schedule_read, bandit_list,
            bandit_statistics,
            subwork_done,
            workflow_do, workflow_new, workflow_list,
            workflow_show_current, workflow_delete,
            bandit_schedule_workflow
        ]
        r['default_tools'] = r['read_tools'] + [
            memory_new, memory_update_string, memory_append_string,
            memory_delete
        ]
        r['all_tools'] = r['default_tools']
        return r

    def _create_agent (self, tools_name='default_tools'):
        tools = self._create_tools()[tools_name]
        # ReactAgentExecutorの準備
        app = create_react_agent(
            self.llm, tools, prompt=self.system_prompt,
            checkpointer=InMemorySaver(),
        )

        return app

    def _filterout_messages2(self):
        self.messages = [
            x for x in self.messages
            if x.id not in self.messages2ids
        ]

    def _summarize_messages(self):
        try:
            cur_top = self.messages[0]
            res = summarize_messages(
                self.messages,
                max_tokens=5000,
                max_summary_tokens=2000,
                running_summary=self.running_summary,
                model=self.llm,
                initial_summary_prompt=INITIAL_SUMMARY_PROMPT,
                existing_summary_prompt=EXISTING_SUMMARY_PROMPT,
                final_prompt=FINAL_SUMMARY_PROMPT,
            )
            self.messages = res.messages
            if self.messages[0].content != cur_top.content:
                print(f"新しい要約: {self.messages[0].content}")
            self.running_summary = res.running_summary
        except Exception as e:
            import traceback
            traceback.print_exc()
            self._sanitize_messages()

    def _sanitize_messages(self):
        print("おかしなエラーが出ているため対処療法として messages をサニタイズします。")
        self.messages = [
            m for m in self.messages
            if not (isinstance(m, AIMessage) and m.tool_calls)
        ]

    def run (self, workflow_main_id):
        print("\n\n----------\n\n")
        self.messages2ids = []

        self.workflow_current = workflow_main_id
        bandits = copy.deepcopy(
            self.workflows[self.workflow_current]['stack']
        )
        working_bandit = None
        workflow_stack = []
        execed = []
        while True:
            while working_bandit is not None or bandits:
                if working_bandit is not None:
                    b, done, prev_done = working_bandit
                    working_bandit = None
                else:
                    b = bandits.pop()
                    done = 0
                    prev_done = True
                enforce = b['tool_name']
                aux_prompt = b['aux_prompt']
                tools_name = b['tools_name']
                memory_id = None
                workflow_id = None
                if b['arg'] and enforce == 'memory_read':
                    memory_id = b['arg']
                if b['arg'] and enforce == 'workflow_do':
                    workflow_id = b['arg']

                while done < b['times']:
                    if not random.random() < b['prob']:
                        done += 1
                        continue
                    if memory_id and memory_id not in self.memories:
                        done += 1
                        continue
                    if workflow_id and workflow_id not in self.workflows:
                        done += 1
                        continue
                    all_tools = [x.name for x in self._create_tools()[tools_name]]
                    tool_names = re.split(r"\s+or\s+|\s+OR\s+", enforce)
                    if not any (x in all_tools for x in tool_names):
                        done += 1
                        continue
                    if memory_id:
                        aux_prompt = f"{memory_id} を読んでください。"
                    if workflow_id:
                        aux_prompt = f"{workflow_id} を実行してください。"
                    self.cur_bandit = b
                    self.cur_bandit_done = done
                    self.cur_bandit_prompt = f"補助にツールをいろいろ使いながら最終的に {enforce} {'のどれか' if ' or ' in enforce.lower() else ''}を適当なパラメータで使ってください。{'(補助プロンプト): ' + aux_prompt if aux_prompt else ''}"
                    prompt = self.cur_bandit_prompt
                    if not prev_done:
                        prompt = "前回の指示がまだ完了してません。前回の指示: " + prompt
                    print(f"USER_INPUT: {prompt}")
                    self.messages.append(HumanMessage(prompt))
                    config = {"configurable": {"thread_id": "1"}}
                    app = self._create_agent(tools_name=tools_name)
                    self.access_unit = 0.3 if memory_id else 1.0
                    prev_done = False
                    try:
                        for chunk, metadata in app.stream(
                                {"messages": self.messages.copy()},
                                config=config,
                                stream_mode="messages",
                        ):
                            if chunk.id in self.messages2ids:
                                continue
                            self.messages.append(chunk)
                            if isinstance(chunk, ToolMessage):
                                print(f"ツール結果({chunk.name}): {short_repr(chunk.content)}")
                            if isinstance(chunk, ToolMessage) \
                               and chunk.name == "workflow_do":
                                if chunk.name in re.split(r"\s+or\s+|\s+OR\s+", enforce) \
                                   and (not workflow_id or workflow_id == self.workflow_next):
                                    done += 1
                                    prev_done = True
                                    execed.append(b)
                                    if not self.workflow_next:
                                        break
                                if not self.workflow_next:
                                    continue
                                workflow_stack.append((
                                    (b, done, prev_done),
                                    bandits,
                                    execed,
                                    self.workflow_current
                                ))
                                self.workflow_current = self.workflow_next
                                bandits = copy.deepcopy(self.workflows[self.workflow_current]['stack'])
                                working_bandit = None
                                execed = []
                                break
                            elif isinstance(chunk, ToolMessage) \
                               and chunk.name in re.split(r"\s+or\s+|\s+OR\s+", enforce) \
                               and (not memory_id or memory_id == self.recent_reads[-1]['id']):
                                done += 1
                                prev_done = True
                                execed.append(b)
                                break
                        self._filterout_messages2()
                        self._summarize_messages()
                        print(f"エージェントの応答: {self.messages[-1].content}")
                    except GraphRecursionError as e:
                        print(f"Recursion Limit に到達しました。")
                        self._filterout_messages2()
                        self._summarize_messages()
                    except Exception as e:
                        print(f"エラーが発生しました: {e}")
                        import traceback
                        traceback.print_exc()
                        self._sanitize_messages()
                        #raise e

            for b in execed:
                for x in self.workflows[self.workflow_current]['stack']:
                    if x['tool_name'] == b['tool_name'] \
                       and x['exec_mode'] == b['exec_mode'] \
                       and x['aux_prompt'] == b['aux_prompt'] \
                       and x['arg'] == b['arg'] \
                       and x['exec_mode'] == "once":
                        if x['times'] > 0:
                            x['times'] -= 1
            self.workflows[self.workflow_current]['stack'] = [
                x for x in self.workflows[self.workflow_current]['stack']
                if x['exec_mode'] != 'once' or x['pin'] or x['times'] > 0
            ]

            if not workflow_stack:
                break
            workflow_prev = self.workflow_current
            prev_title = self.workflows[workflow_prev]['title']
            working_bandit, bandits, execed, self.workflow_current \
                = workflow_stack.pop()
            cur_title = self.workflows[self.workflow_current]['title']
            mes = f"{workflow_prev} 「{prev_title}」から {self.workflow_current} 「{cur_title}」に復帰しました。"
            print(f"USER_INPUT: {mes}")
            self.messages.append(HumanMessage(mes))

    def listen_and_print (self, prompt):
        ans = None
        try:
            app = self._create_agent(tools_name='null_tools')
            config = {"configurable": {"thread_id": "1"}}
            print(f"USER_INPUT: {prompt}")
            response = app.invoke(
                {"messages": self.messages + [HumanMessage(prompt)]},
                config=config
            )
            self.messages = response['messages']
            self._summarize_messages()
            ans = response['messages'][-1].content
            print(f"エージェントの応答: {ans}")
        except Exception as e:
            print(f"エラーが発生しました: {e}")
        print("")
        sleep(3)
        return ans

    def init_memories (self):
        memories = [
            {
                'id': 'memory:9998',
                'title': 'メモリの文書を検索する手順',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                まずどういうものが探したいか、express_thought してみる。

                その上でそれにまつわるキーワードを imagine_keywords で連想してみる。

                それらにしたがって memory_words_search や memory_semantic_search をやってみる。
                """)
            },
            {
                'id': 'memory:9997',
                'title': 'メモリに文書を残す手順',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                行動結果や得た知識を積極的にメモリに書き残していこう。

                メモリに書くときは次のような要素を使う。

                [memory:〜] : メモリID への参照を明示する。
                keyword:〜 : そのメモリにまつわるキーワードを指定する。

                キーワードは将来のメモリへも実質的にリンクできることに注意しよう。

                例:

                私が [memory:5555] に従って歩いていると確かに妖怪に出くわした。

                keyword: 妖怪

                おそろしかった。
                """)
            },
            {
                'id': 'memory:9995',
                'title': 'ツールが実行できない！と思ったら',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                指示に無関係なツールは実行できないことがある。現在使えるツールをよく確かめよう。
                """)
            },
            {
                'id': 'memory:9994',
                'title': 'keyword の増強',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                memory_list_random という機能を使って 5 つリストアップして、それぞれを読み、それぞれに適切なキーワードを付けることが可能なら 「keyword: 〜」文を append するように。
                """)
            }
        ]
        for x in memories:
            self.update_keywords(x['text'])
            self.memories[x['id']] = x
            self.update_vector(x)

    def init_workflows (self):
        workflow_main = [
            {
                'tool_name': 'memory_new',
                'tools_name': 'default_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "最近のやり取りを要約して書いてください。",
                'arg': None,
                'prob': 0.1,
                'times': 1,
                'pin': 'stack'
            },
            # {
            #     'tool_name': 'subwork_done',
            #     'tools_name': 'default_tools',
            #     'exec_mode': 'persistent',
            #     'aux_prompt': "ワークフローのテストをします。適当なパラメータで workflow_new を実行し、それをすぐさま workflow_delete します。そして、bandit_schedule で、指示しているこのバンディットを削除します。完了したら subwork_done してください。",
            #     'arg': None,
            #     'prob': 1.0,
            #     'times': 1,
            #     'pin': None,
            # },
            {
                'tool_name': 'memory_new OR memory_update_string OR memory_append_string',
                'tools_name': 'default_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': None,
                'prob': 0.4,
                'times': 1,
                'pin': 'stack'
            },
            {
                'tool_name': 'workflow_do',
                'tools_name': 'default_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': "workflow:1000",
                'prob': 1.0/20,
                'times': 1,
                'pin': 'stack'
            },
            {
                'tool_name': 'memory_read',
                'tools_name': 'default_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': None,
                'prob': 0.5,
                'times': 3,
                'pin': 'stack'
            },
            {
                'tool_name': 'memory_read',
                'tools_name': 'read_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': 'memory:9998',
                'prob': 0.1,
                'times': 1,
                'pin': None
            },
            {
                'tool_name': 'memory_read',
                'tools_name': 'read_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': 'memory:9997',
                'prob': 0.1,
                'times': 1,
                'pin': None
            },
        ]
        self.workflows["workflow:main"] \
            = {'pin': 'de', 'stack': workflow_main, 'title': "メイン"}

        workflow_sub = [
            {
                'tool_name': 'subwork_done',
                'tools_name': 'default_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "memory:9994 を読みその指示を実行し、そのタスクが終ったら、subwork_done してください。",
                'arg': None,
                'prob': 1.0,
                'times': 1,
                'pin': 'write'
            },
            {
                'tool_name': 'memory_read',
                'tools_name': 'read_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': 'memory:9994',
                'prob': 1.0,
                'times': 1,
                'pin': 'write'
            }
        ]
        self.workflows["workflow:1000"] \
            = {'pin': 'wd', 'stack': workflow_sub, 'title': 'keyword 更新'}

    def update_keywords (self, text):
        extracted_keywords = []

        pattern1 = r'keyword:\s*(.*?)(?:\n|$)'
        matches1 = re.findall(pattern1, text, re.IGNORECASE)
        extracted_keywords.extend([kw.strip() for kw in matches1])

        pattern2 = r'\[keyword:\s*(.*?)\]'
        matches2 = re.findall(pattern2, text, re.IGNORECASE)
        extracted_keywords.extend([kw.strip() for kw in matches2])

        for keyword in extracted_keywords:
            if keyword.startswith("〜"):
                continue
            if keyword and keyword not in self.keywords:
                self.keywords.append(keyword)

    def update_vector (self, x):
        text = x['title'] + "\n\n" + x['text']
        x['vector'] = self.emb_llm.embed_query(text)

    def _create_backend_agent (self):
        @tool
        def set_result (status: str, result: Any)  -> None:
            """
            結果を設定します。

            Args:
            status (str): 処理のステータス ('success' または 'error')。
            result (Any): 処理結果のデータまたはエラーメッセージ。
            """
            print(f"ツール2(set_result): status: {repr(status)}, result: {short_repr(result)}")
            self.backend_status = status
            self.backend_result = result

        @tool
        def read_all_memories ()  -> Dict[str, Any]:
            """
            全ての記憶を読み込みます。
            """
            print("ツール2(read_all_memories):...")
            return {'status': 'success',
                    'result': [{k: v for k, v in x.items() if k != 'vector'}
                               for x in self.memories.values()]}

        @tool
        def read_all_keywords ()  -> Dict[str, Any]:
            """
            全てのキーワードを読み込みます。
            """
            print("ツール2(read_all_keywords):...")
            return {'status': 'success',
                    'result': [x for x in self.keywords]}

        @tool
        def express_thought (thought: str) -> None:
            """
            プレイヤーの現在の考えを吐露します。
            """
            mes = f"「{thought}」と考えが吐露されました。"
            print(f"ツール2(express_thought): {mes}")

        tools = [set_result, read_all_memories, read_all_keywords,
                 express_thought]

        # ReactAgentExecutorの準備
        app = create_react_agent(
            self.llm2, tools, prompt=self.system_prompt2,
            checkpointer=InMemorySaver(),
        )

        return app

    def call_backend_agent (self, user_input):
        config = {"configurable": {"thread_id": "2"}}
        app = self._create_backend_agent()
        self.messages2 = []
        self.backend_result = None
        self.backend_status = None
        while self.backend_result is None or self.backend_status is None:
            try:
                sleep(3)
                print(f"USER_INPUT2: {user_input}")
                self.messages2.append(HumanMessage(user_input))
                for chunk, metadata in app.stream(
                        {"messages": self.messages2.copy()},
                        config=config,
                        stream_mode="messages",
                ):
                    self.messages2.append(chunk)
                    self.messages2ids.append(chunk.id)
                    if isinstance(chunk, ToolMessage):
                        print(f"ツール結果2({chunk.name}): {short_repr(chunk.content)}")
                    if isinstance(chunk, ToolMessage) and chunk.name == "set_result":
                        break
                print(f"エージェントの応答: {self.messages2[-1].content}")
            except GraphRecursionError as e:
                print(f"Recursion Limit に到達しました。")
            except Exception as e:
                print(f"エラーが発生しました: {e}")
                import traceback
                traceback.print_exc()
                #raise e

            sleep(3)

        return {'status': self.backend_status, 'result': self.backend_result}


メインの RagAgent。

In [11]:
class RagAgent (MemoryBanditWorkflow):
    def __init__ (self, llm=llm, llm2=llm, emb_llm=emb_llm,
                  save_file=None):
        self.thesis = {
            'title': "",
            'chapters': [{'title': '概要', 'text': ""}]
        }
        super().__init__(llm=llm, llm2=llm, emb_llm=emb_llm,
                         save_file=save_file)

        self.system_prompt = """\
あなたは賢い RAG エージェントです。論文(thesis)を書き上げていきます。
利用可能なツールを使用して、迷路をナビゲートし、ゴールに到達してください。
現在の計画と方針と周囲の状況を考慮し、必要に応じて計画と方針を更新してください。
あなたとは別の者が次の行動をしやすいよう計画と方針を残してください。

メモリの id は memory:～ という形式です。memory:5555 を指定するときは 5555 だけではダメです。文の中でメモリの参照を示すときは [memory:〜] のように書きます。
「全地図と地図記号」は [memory:9999] にあります。「メモリの文書を検索する手順」は [memory:9998] にあります。「メモリに文書を残す手順」は [memory:9997] にあります。

論文(thesis)は章(chapter)に分けて作っていきます。それぞれが Markdown 形式です。論文のタイトルは # 一つ、章のタイトルは ## で修飾されることになりますので、それに合わせた階層で書いてください。
"""

        self.system_prompt2 = """\
あなたは賢い RAG エージェントをサポートする裏方のエージェントです。
本来この裏方は様々な技法を用いて実装される予定なのですが、現在は試験実装中のため、その動きをあなたは偽装しなければなりません。

よく考えツールを積極的に使って Human からの指示に従ってください。
"""
        self.subwork_done = True
        self.user_demand = ''
        self.current_work = ''
        self.current_state = 0

    def _create_tools (self):
        @tool
        def show_user_demand () -> str:
            """
            大目的を返します。
            """
            print(f"ツール(show_user_demand): {self.user_demand}")
            return self.user_demand

        @tool
        def show_current_work () -> str:
            """
            中目的すなわち現在の仕事を返します。
            """
            print(f"ツール(show_current_work): {self.current_work}")
            return self.current_work

        @tool
        def thesis_write_title (new_title: str) -> str:
            """
            論文のタイトルを更新します。
            """
            self.thesis['title'] = new_title
            mes = "論文のタイトルが更新されました。"
            print(f"ツール(thesis_write_title): {mes}: {new_title}")
            return mes

        @tool
        def thesis_show_title () -> str:
            """
            論文のタイトルを返します。
            """
            print(f"ツール(thesis_show_title): {self.thesis['title']}")
            return self.thesis['title']

        @tool
        def thesis_new_chapter (title: str, text: str) -> str:
            """
            新しい章を title と text で構成します。
            """

            x = {'title': title, 'text': text}
            num = len(self.thesis['chapters'])
            self.thesis['chapters'].append(x)
            print(f"ツール(thesis_new_chapter): {short_repr(x)}")
            return f"成功: 第{num}章を作りました。"

        @tool
        def thesis_write_chapter (
                chapter_num: int,
                new_title: str,
                new_text: str
        ) -> str:
            """
            章を new_title と new_text で置き換えます。
            """

            if not (chapter_num >= 0
                    and chapter_num < len(self.thesis['chapters'])):
                return f"失敗: 第{chapter_num}章は現在ありません。"
            x = {'title': new_title, 'text': new_text}
            self.thesis['chapters'][chapter_num] = x
            print(f"ツール(thesis_write_chapter): {short_repr(x)}")
            return f"成功: 第{chapter_num}章を書き換えました。"

        @tool
        def thesis_delete_last_chapter () -> str:
            """
            最後の章を削除します。
            """
            num = len(self.thesis['chapters']) - 1
            if num <= 0:
                return f"失敗: これ以上削除できません。"
            self.thesis['chapters'].pop()
            return f"成功: 第{num}章を削除しました。"

        @tool
        def thesis_read_chapter(chapter_num: int) -> Union[Dict[str, str], str]:
            """
            指定された章を読み込みます。
            """

            if not (chapter_num >= 0
                    and chapter_num < len(self.thesis['chapters'])):
                return f"失敗: 第{chapter_num}章は現在ありません。"
            return self.thesis['chapters'][chapter_num]

        @tool
        def thesis_list_chapters() -> List[str]:
            """
            章のタイトルのリストを返します。
            """
            return [x['title'] for x in self.thesis['chapters']]

        @tool
        def subwork_done()  -> str:
            """
            指示されたサブワークが(成功裡に)完了したことを宣言します。
            """
            self.subwork_done = True
            return f"成功。サブワークの完了が宣言されました。"

        @tool
        def subwork_not_done()  -> str:
            """
            指示されたサブワークが完了してないことを宣言します。
            """
            self.subwork_done = False
            return f"成功。サブワークの未完了が宣言されました。"

        @tool
        def show_execution () -> str:
            """
            workflow より上位の実行制御の全体像とそこにおける現在の State を返します。
            """
            mes = dedent(f"""\
            大目的は「{self.user_demand}」です。

            次に実行制御の全体像を示します。State 0 からはじまります。指示があったとき subwork_done または subwork_not_done することで次の State に遷移します。

            State 0: 大目的に沿ってメモリ等に計画を立てます。完了時 State 1 へ。

            State 1: 大目的に沿ってウェブを調査し取材してまだ書いていない一章分ずつメモリ等に記録していきます。完了時 State 2 へ。

            State 2: 大目的に沿ってメモリ等にある取材データが全章分、十分か確かめます。完了時 State 3 へ。未完了時 State 1 へ。

            State 3: 大目的に沿ってメモリ等にある取材データを元に論文(thesis)をまだ書いてない一章分、十分でない一章分を書きます。完了時 State 4 へ。

            State 4: 大目的に沿って取材データを元にした論文(thesis)が全章完成しているかをチェックします。最後にタイトルを設定し、第0章の概要を書き直します。完了時 State 6 へ。未完了時 State 5 へ。

            State 5: 大目的に沿ってまだ追加取材が必要なら再調査してメモリ等に記録します。完了時 State 3 へ

            State 6: 終了。完成！

            現在の State は {self.current_state} です。
            """)
            print(f"ツール(show_execution): {mes}")
            return mes


        @tool
        def get_web_page_content(url: str) -> str:
            """
            指定されたURLからWebページのコンテンツを取得して文字列として返します。
            調査でURLを見つけた後に使ってください。
            """
            try:
                loader = WebBaseLoader(url)
                docs = loader.load()
                content = " ".join([doc.page_content for doc in docs])
                print(f"ツール(get_web_page_content): 成功: {url}")
                # 長すぎる場合は適当に丸める
                if len(content) > 5000:
                    return content[:5000] + "... (コンテンツが長すぎるため省略)"
                return content
            except Exception as e:
                print(f"ツール(get_web_page_content): 失敗: {url}")
                return f"エラー: ページの読み込みに失敗しました。URL: {url}, エラー: {e}"

        search_tool = DuckDuckGoSearchResults(output_format="list", verbose=True)
        web_page_tool = get_web_page_content

        r = super()._create_tools()
        for k in r:
            if k != 'null_tools':
                r[k] = self._replace_tools(r[k], [
                    show_user_demand, show_current_work,
                    thesis_write_title, thesis_show_title,
                    thesis_new_chapter, thesis_write_chapter,
                    thesis_delete_last_chapter,
                    thesis_read_chapter, thesis_list_chapters,
                    subwork_done, subwork_not_done, show_execution,
                    search_tool, web_page_tool
                ])
        return r

    def main_loop (self, user_demand):
        self.user_demand = user_demand
        self.current_state = 0
        self.resume()

    def resume (self):
        if self.current_state == 0:
            self.execute_planning()
            self.current_state = 1
            self.save()
            print("\n\n----------\n\n")
        while self.current_state in [1, 2]:
            if self.current_state == 1:
                self.execute_investigation()
                self.current_state = 2
                self.save()
                print("\n\n----------\n\n")
            if self.current_state == 2:
                r = self.execute_check_of_investigation()
                if r:
                    self.current_state = 3
                else:
                    self.current_state = 1
                self.save()
                print("\n\n----------\n\n")
        while self.current_state in [3, 4, 5]:
            if self.current_state == 3:
                r = self.execute_writing()
                self.current_state = 4
                self.save()
                print("\n\n----------\n\n")
            if self.current_state == 4:
                r = self.execute_check_of_writing()
                if r:
                    self.current_state = 6
                else:
                    self.current_state = 5
                self.save()
                print("\n\n----------\n\n")
            if self.current_state == 5:
                self.execute_reinvestigation()
                self.current_state = 3
                self.save()
                print("\n\n----------\n\n")
        print("Done!")

    def write_loop (self):
        if self.current_state not in [3, 4, 5]:
            self.current_state = 3
        self.resume()

    def _execute (self, current_work, tool_name, aux_prompt):
        self.current_work = current_work
        user_input = dedent(f"""\
        大目的: 「{self.user_demand}」
        計画と方針: 「{self.plan}」
        スクラッチパッド: 「{self.scratchpad}」
        すでに必要なものが書かれていないか確かめながら進みましょう。
        中目的または現在の仕事: 「{self.current_work}」
        """)
        print(f"USER_INPUT: {user_input}")
        self.messages.append(HumanMessage(user_input))

        self.workflows['workflow:main']['stack'][0] = {
            'tool_name': tool_name,
            'tools_name': 'all_tools',
            'exec_mode': 'persistent',
            'aux_prompt': aux_prompt,
            'arg': None,
            'prob': 1.0,
            'times': 1,
            'pin': 'write'
        }
        self.run("workflow:main")

        return self.subwork_done

    def execute_planning (self):
        return self._execute(
            current_work = "大目的に沿ってメモリ等に計画を立ててください。",
            tool_name = 'subwork_done',
            aux_prompt = "計画を立て終ったら subwork_done してください。",
        )

    def execute_investigation (self):
        return self._execute(
            current_work = "大目的に沿ってウェブを調査し取材してまだ書いていない一章分ずつメモリ等に記録していってください。",
            tool_name = 'subwork_done',
            aux_prompt = "一章分取材し終ったら subwork_done してください。",
        )

    def execute_check_of_investigation (self):
        return self._execute(
            current_work = "大目的に沿ってメモリ等にある取材データが全章分、十分か確かめてください。",
            tool_name = 'subwork_done OR subwork_not_done',
            aux_prompt = "全章の取材が十分なら subwork_done してください。十分でないと判断したら subwork_not_done してください。",
        )

    def execute_writing (self):
        return self._execute(
            current_work = "大目的に沿ってメモリ等にある取材データを元に論文(thesis)をまだ書いてない一章分、十分でない一章分を書いてください。ちなみに第0章は概要と決まっています。",
            tool_name = 'subwork_done',
            aux_prompt = "良い論文のうち一章分書けたら subwork_done してください。",
        )

    def execute_check_of_writing (self):
        return self._execute(
            current_work = "大目的に沿って取材データを元にした論文(thesis)が全章完成しているかをチェックしてください。最後にタイトルを設定し、第0章の概要を書き直してください。",
            tool_name = 'subwork_done OR subwork_not_done',
            aux_prompt = "良い論文が全章書け、タイトルを設定し、概要を書き直せたら subwork_done してください。まだ全章書けていなければ subwork_not_done してください。",
        )

    def execute_reinvestigation (self):
        return self._execute(
            current_work = "大目的に沿ってまだ追加取材が必要なら再調査してメモリ等に記録してください。",
            tool_name = 'subwork_done',
            aux_prompt = "取材し終ったら subwork_done してください。",
        )

    def init_memories (self):
        super().init_memories()
        memories = []
        for x in memories:
            self.update_keywords(x['text'])
            self.memories[x['id']] = x
            self.update_vector(x)

    def init_workflows (self):
        super().init_workflows()
        workflow_main = self.workflows['workflow:main']['stack']
        workflow_main = [
            {
                'tool_name': 'subwork_done',
                'tools_name': 'all_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "仮登録。",
                'arg': None,
                'prob': 1.0,
                'times': 1,
                'pin': 'write'
            }
        ] + workflow_main
        self.workflows['workflow:main']['stack'] = workflow_main


    def display_thesis (self):
        s = f"# {self.thesis['title']}\n\n"
        for i, x in enumerate(self.thesis['chapters']):
            if re.search(r"^\#\#", x['text']):
                pass
            elif i == 0 or re.search(r"[01-9]+章", x['title']):
                s += f"## {x['title']}\n\n"
            else:
                s += f"## 第{i}章 {x['title']}\n\n"
            s += f"{x['text']}\n\n"
        return Markdown(s)


クラスをインスタンシエート。

In [20]:
agent = RagAgent(llm=llm, llm2=llm, emb_llm=emb_llm, save_file=RAG_AGENT_SAVE)
agent.save()

途中から始める場合は直前のコードを実行せず、次だけを実行します。

In [ ]:
agent = RagAgent.load(RAG_AGENT_SAVE, llm=llm, llm2=llm, emb_llm=emb_llm)

では、動かしてみます。

In [21]:
agent.main_loop("日本の失われたウン十年、何をすれば良かったかを論文にまとめてください。")

USER_INPUT: 大目的: 「日本の失われたウン十年、何をすれば良かったかを論文にまとめてください。」
計画と方針: 「まだ計画と方針はセットされていません。」
スクラッチパッド: 「」
すでに必要なものが書かれていないか確かめながら進みましょう。
中目的または現在の仕事: 「大目的に沿ってメモリ等に計画を立ててください。」



----------


USER_INPUT: 補助にツールをいろいろ使いながら最終的に memory_read を適当なパラメータで使ってください。
ツール(update_plan): 計画と方針が更新されました。: 1. **情報収集**: ウェブ検索ツールを使い、「日本の失われた数十年」の原因、経緯、対策について調査する。
2. **構成案作成**: 収集した情報を基に、論文の章立てと各章で記述する内容の骨子を作成する。
3. **計画の保存**: 作成した構成案と今後の作業計画をメモリに保存する。
4. **作業開始**: 保存した計画に従い、論文の執筆を開始する。
ツール結果(update_plan): '計画と方針が更新されました。'
content='[{"snippet": "3．斎藤ジン氏の指摘とその背景 斎藤ジン氏は『世界秩序が変わるとき』において、日本が「失われた 30 年」を迎えた主な原因として、世界的な経済構造の変化に適応できなかったことを挙げています。", "title": "日本の失われた30年──都合よく利用される停滞の物語と違和感", "link": "https://note.com/sketto10/n/n9d755cdbcb8d"}, {"snippet": "コラム 少子高齢化 失われた 30 年 バブル経済 DX 働き方改革 失われた 30 年とは？ —日本経済の長期停滞の原因と影響 「失われた30年」とは、1990年代初頭のバブル経済崩壊以降、日本経済が長期にわたり低成長、デフレーション、停滞に苦しんだ期間を指します。", "title": "失われた30年とは？—日本経済の長期停滞の原因と影響 - K2 College", "link": "https://media.k2-assurance.com/archives/28408"}, {"snippet": "Apr 24, 20

/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:2007: UserWarning: HumanMessage with empty content was removed to prevent API error
  warnings.warn(


新しい要約: これまでの会話の要約: はい、承知いたしました。これまでの会話の要約です。

**目的:**
ユーザーは「日本の失われたウン十年、何をすれば良かったか」というテーマで論文を作成するよう指示しました。

**実行内容:**
1.  **計画立案と情報収集:**
    *   まず、論文作成のための計画（情報収集 → 構成案作成 → 保存 → 執筆）を立てました。
    *   ウェブ検索ツール(`duckduckgo_results_json`)を使い、「失われた30年」の原因（不良債権処理の遅れ、経済構造の変化など）について調査しました。

2.  **構成案の作成と保存:**
    *   収集した情報に基づき、序論から結論までの論文構成案を作成しました。
    *   作成した構成案をメモリツール(`memory_new`)で保存し、今後の作業指針としました。
    *   ユーザーの指示に従い、計画立案が完了したことを報告(`subwork_done`)しました。

3.  **論文執筆の開始:**
    *   論文執筆に着手し、再度ウェブ検索で序論に必要な情報を収集しました。
    *   収集した情報（バブル崩壊、金融政策の遅れなど）を基に、論文の「序論」を執筆し、論文作成ツール(`thesis_new_chapter`)に保存しました。

**現在の状況:**
論文の構成案が完成し、それに基づいて「序論」の執筆が完了しました。次は第1章の執筆に進む段階です。
エージェントの応答: {"id": "memory:1000", "title": "論文「日本の失われた30年、何をすれば良かったか」構成案", "accesses": 2.0, "text": "# 論文構成案\n\n## 序論\n- 「失われた30年」の定義と問題提起\n\n## 第1章: バブル経済の崩壊と不良債権問題\n- バブル発生の経緯\n- 不良債権処理の遅れとその影響\n\n## 第2章: グローバル化と産業構造の変化への対応の遅れ\n- 製造業の空洞化\n- デジタル化への対応の遅れ\n- イノベーションの停滞\n\n## 第3章: デフレと金融政策\n- デフレスパイラルの影響\n- 金融・財政政策の評価\n\n## 第4章: 人口減少と社会構造の

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py", line 208, in _chat_with_retry
    return generation_method(**kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/ai/generativelanguage_v1beta/services/generative_service/client.py", line 1183, in stream_generate_content
    response = rpc(
               ^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/gapic_v1/method.py", line 131, in __call__
    return wrapped_func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_unary.py", line 294, in retry_wrapped_func
    return retry_target(
           ^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_unary.py", line 156, in retry_target
    next_sleep = _retry_error_helper(
                 ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/l

USER_INPUT2: メモリ全部を search_str = '論文の構成案' でセマンティックにサーチするのを偽装してください。

ただし、メモリは read_all_memories や ツールで得られる本物のメモリを使ってください。

set_result ツールで結果を返してください。

status は 'error' か 'success'。
result は「マッチデータ」のリスト。
マッチデータはそれぞれが辞書型、それを m とすると。
m['id'] はメモリ id。memory:〜 という形式。
m['title'] はメモリの title。
m['snippet'] はメモリの text のそのセマンティックにマッチする部分周辺。

ツール2(read_all_memories):...
ツール結果(read_all_memories): '{"status": "success", "result": [{"id": "memory:9998", "title": "メモリの文書を検索す...'
ツール結果2(read_all_memories): '{"status": "success", "result": [{"id": "memory:9998", "title": "メモリの文書を検索す...'
ツール2(set_result): status: 'success', result: [{'title': '論文「日本の失われた30年、何をすれば良かったか」構成案', 'id': 'memory:1000', 'snippet': '...]
ツール結果2(set_result): 'null'
エージェントの応答: null
ツール結果(memory_semantic_search): '{"status": "success", "result": [{"title": "論文「日本の失われた30年、何をすれば良かったか」構成案", ...'
ツール結果(memory_read): '{"id": "memory:1000", "title": "論文「日本の失われた30年、何をすれば良かったか」構成案", "accesses": ...'
エージェントの応答: {"id": "memory:1000", "title":

...

Traceback (most recent call last):
  File "/tmp/ipython-input-4174715272.py", line 818, in _summarize_messages
    res = summarize_messages(
          ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langmem/short_term/summarization.py", line 445, in summarize_messages
    preprocessed_messages = _preprocess_messages(
                            ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langmem/short_term/summarization.py", line 174, in _preprocess_messages
    raise ValueError(
ValueError: Message with ID run--1ceabdd2-5a50-44f0-a6eb-acf5d454567e has already been summarized.


ツール結果(memory_read): '{"id": "memory:1010", "title": "序論（草稿）", "accesses": 6.1, "text": "## 序論\\n...'
ツール結果(memory_read): '{"id": "memory:1007", "title": "第1章「バブル経済の崩壊と不良債権問題」取材メモ", "accesses": 5.1,...'
ツール結果(memory_read): '{"id": "memory:1008", "title": "第2章「グローバル化と産業構造の変化への対応の遅れ」取材メモ", "accesses"...'
ツール結果(memory_read): '{"id": "memory:1003", "title": "第3章「デフレと金融政策」取材メモ", "accesses": 5.2, "text"...'
ツール結果(memory_read): '{"id": "memory:1004", "title": "第4章「人口減少と社会構造の変化」取材メモ", "accesses": 5.1, "t...'
ツール結果(memory_read): '{"id": "memory:1012", "title": "第5章取材メモ: 「失われた30年」からの教訓", "accesses": 2.1, ...'
ツール結果(memory_read): '{"id": "memory:1013", "title": "第5章取材メモ: 構造改革と具体的な政策提言", "accesses": 1.1, "...'
ツール(express_thought): 「論文全体の構成と各章の要点を再確認した。序論から第5章までのメモは揃っている。計画通り、これらの要点を踏まえて「結論」を執筆する。

**結論の構成案:**
1.  **「失われた30年」の総括:**
    *   バブル崩壊後の不良債権処理の遅れが全ての起点であったことを指摘。
    *   金融・財政政策の迷走がデフレを深刻化させた。
    *   グローバル化やデジタル化といった構造変化への対応が遅れ、産業競争力を失った。
    *   少子高齢化という静かなる危機が、経済の足かせとなり続けた。
2.  **「何をすれば

Traceback (most recent call last):
  File "/tmp/ipython-input-4174715272.py", line 818, in _summarize_messages
    res = summarize_messages(
          ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langmem/short_term/summarization.py", line 445, in summarize_messages
    preprocessed_messages = _preprocess_messages(
                            ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langmem/short_term/summarization.py", line 174, in _preprocess_messages
    raise ValueError(
ValueError: Message with ID run--1ceabdd2-5a50-44f0-a6eb-acf5d454567e has already been summarized.


USER_INPUT: 補助にツールをいろいろ使いながら最終的に subwork_done OR subwork_not_done のどれかを適当なパラメータで使ってください。(補助プロンプト): 良い論文が全章書け、タイトルを設定し、概要を書き直せたら subwork_done してください。まだ全章書けていなければ subwork_not_done してください。
ツール結果(memory_read): '{"id": "memory:1010", "title": "序論（草稿）", "accesses": 7.1, "text": "## 序論\\n...'
ツール結果(memory_read): '{"id": "memory:1007", "title": "第1章「バブル経済の崩壊と不良債権問題」取材メモ", "accesses": 6.1,...'
ツール結果(memory_read): '{"id": "memory:1008", "title": "第2章「グローバル化と産業構造の変化への対応の遅れ」取材メモ", "accesses"...'
ツール結果(memory_read): '{"id": "memory:1003", "title": "第3章「デフレと金融政策」取材メモ", "accesses": 6.2, "text"...'
ツール結果(memory_read): '{"id": "memory:1004", "title": "第4章「人口減少と社会構造の変化」取材メモ", "accesses": 6.1, "t...'
ツール結果(memory_read): '{"id": "memory:1012", "title": "第5章取材メモ: 「失われた30年」からの教訓", "accesses": 3.1, ...'
ツール(express_thought): 「論文全体の構成を再確認するために、序論[memory:1010]、第1章[memory:1007]、第2章[memory:1008]、第3章[memory:1003]、第4章[memory:1004]、第5章[memory:1012]のメモを読み返した。
各章の要点は以下の通り。

*   **序論**: 「失われた30年」の定義と問題提起。
*   **第1章**:

Traceback (most recent call last):
  File "/tmp/ipython-input-4174715272.py", line 818, in _summarize_messages
    res = summarize_messages(
          ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langmem/short_term/summarization.py", line 445, in summarize_messages
    preprocessed_messages = _preprocess_messages(
                            ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langmem/short_term/summarization.py", line 174, in _preprocess_messages
    raise ValueError(
ValueError: Message with ID run--1ceabdd2-5a50-44f0-a6eb-acf5d454567e has already been summarized.


おかしなところで終了してしまいました。途中から再び書くように命じてみます。その前に summarize_message 周りにエラーに対処しておきます。

In [22]:
agent.running_summary = None

途中から再び書くように命じます。

In [23]:
agent.write_loop()

USER_INPUT:         大目的: 「日本の失われたウン十年、何をすれば良かったかを論文にまとめてください。」
        計画と方針: 「1.  **現状の再確認:** 序論から第5章までの本文を構成する全ての章について、草稿または取材メモが揃っていることを確認済み。

2.  **最終執筆計画の立案:**
    *   残る作業である「結論」の執筆と、論文全体の推敲・編集に関する具体的な計画を立てる。

3.  **結論の執筆:**
    *   全体の論旨をまとめるため、序論から第5章までの草稿および取材メモを全て読み返す。
    *   上記を踏まえ、「結論」の草稿を執筆し、新しいメモリに保存する。

4.  **推敲と完成:**
    *   全章の草稿を読み返し、論理構成、表現、誤字脱字などを修正し、論文全体を完成させる。」
        スクラッチパッド: 「」
        すでに必要なものが書かれていないか確かめながら進みましょう。
        中目的または現在の仕事: 「大目的に沿ってメモリ等にある取材データを元に論文(thesis)をまだ書いてない一章分、十分でない一章分を書いてください。ちなみに第0章は概要と決まっています。」



----------


USER_INPUT: 補助にツールをいろいろ使いながら最終的に memory_read を適当なパラメータで使ってください。(補助プロンプト): memory:9997 を読んでください。
ツール結果(memory_read): '{"id": "memory:1010", "title": "序論（草稿）", "accesses": 8.4, "text": "## 序論\\n...'
ツール結果(memory_read): '{"id": "memory:1007", "title": "第1章「バブル経済の崩壊と不良債権問題」取材メモ", "accesses": 6.39...'
ツール結果(memory_read): '{"id": "memory:1008", "title": "第2章「グローバル化と産業構造の変化への対応の遅れ」取材メモ", "accesses"...'
ツール結果(memory_read): '{"id": "memor

Traceback (most recent call last):
  File "/tmp/ipython-input-4174715272.py", line 906, in run
    for chunk, metadata in app.stream(
                           ^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langgraph/pregel/main.py", line 2647, in stream
    for _ in runner.tick(
             ^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langgraph/pregel/_runner.py", line 253, in tick
    _panic_or_proceed(
  File "/usr/local/lib/python3.12/dist-packages/langgraph/pregel/_runner.py", line 511, in _panic_or_proceed
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/langgraph/pregel/_executor.py", line 81, in done
    task.result()
  File "/usr/lib/python3.12/concurrent/futures/_base.py", line 449, in result
    return self.__get_result()
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/concurrent/futures/_base.py", line 401, in __get_result
    raise self._exception
  File "/usr/lib/python3.12/concurrent/futures/thread.py", line 59, in r

ツール結果(memory_read): '{"id": "memory:1013", "title": "第5章取材メモ: 構造改革と具体的な政策提言", "accesses": 2.4000...'
エージェントの応答: {"id": "memory:1013", "title": "第5章取材メモ: 構造改革と具体的な政策提言", "accesses": 2.4000000000000004, "text": "## 第5章取材メモ: 構造改革と具体的な政策提言\n\n### 1. 規制緩和\n- **目的:** 企業の新規参入や競争を促進し、経済の活性化を図る。\n- **具体例:**\n    - 労働市場の流動化（解雇規制の緩和、非正規雇用の拡大）\n    - 金融ビッグバン（金融機関の参入障壁の撤廃、業務範囲の拡大）\n    - 放送・通信分野の規制緩和\n- **課題:** 雇用の不安定化、格差の拡大\n\n### 2. イノベーションの促進\n- **目的:** 新たな産業や技術を創出し、国際競争力を高める。\n- **具体例:**\n    - 研究開発への投資拡大\n    - ベンチャー企業の育成支援\n    - 産学官連携の強化\n- **課題:** 基礎研究の軽視、短期的な成果の追求\n\n### 3. グローバル化への対応\n- **目的:** 国際的な競争環境に適応し、新たな成長機会を獲得する。\n- **具体例:**\n    - TPP（環太平洋パートナーシップ協定）などの自由貿易協定への参加\n    - 対内直接投資の促進\n    - 企業の海外展開支援\n- **課題:** 国内産業の空洞化、食料自給率の低下\n\n### 4. 人口減少・少子高齢化への対応\n- **目的:** 労働力不足を解消し、社会保障制度を持続可能なものにする。\n- **具体例:**\n    - 女性や高齢者の就労促進\n    - 外国人労働者の受け入れ拡大\n    - 子育て支援の充実\n- **課題:** 社会保障給付と負担のバランス、移民政策に関する国民的合意の形成", "modified_at": "2025-09-24T09:14:41.722517"}
USER_INPUT: 補助にツールをいろいろ使いながら最終的に sub

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py", line 208, in _chat_with_retry
    return generation_method(**kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/ai/generativelanguage_v1beta/services/generative_service/client.py", line 1183, in stream_generate_content
    response = rpc(
               ^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/gapic_v1/method.py", line 131, in __call__
    return wrapped_func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_unary.py", line 294, in retry_wrapped_func
    return retry_target(
           ^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_unary.py", line 156, in retry_target
    next_sleep = _retry_error_helper(
                 ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/l

ツール結果(memory_read): '{"id": "memory:1003", "title": "第3章「デフレと金融政策」取材メモ", "accesses": 9.6, "text"...'
ツール(thesis_new_chapter): {'title': '第3章: デフレと金融政策', 'text': '## 第3章: デフレと金融政策\n\n### 長期デフレの罠\n\nバブル崩壊...}
ツール結果(thesis_new_chapter): '成功: 第7章を作りました。'
USER_INPUT2: メモリ全部を search_str = '第4章 OR 少子高齢化 OR 社会保障制度' で文字列検索するのを偽装してください。OR や () が使えることになっています。

ただし、メモリやキーワードは read_all_memories や read_all_keywords ツールで得られる本物のメモリやキーワードを使ってください。

set_result ツールで結果を返してください。

status は 'error' か 'success'。
result は「マッチデータ」のリスト。
マッチデータはそれぞれが辞書型、それを m とすると。
m['id'] はメモリ id。memory:〜 という形式。
m['title'] はメモリの title。
m['snippet'] はメモリの text のその文字列にマッチする部分周辺。

ツール2(read_all_keywords):...
ツール2(read_all_memories):...
ツール結果2(read_all_memories): '{"status": "success", "result": [{"id": "memory:9998", "title": "メモリの文書を検索す...'
ツール結果(read_all_keywords): '{"status": "success", "result": ["妖怪", "バブル経済, 不良債権, プラザ合意, 総量規制, 金融危機"]}'
ツール結果2(read_all_keywords): '{"status": "success", "result": ["妖怪", "バブル経済, 不良債権, プラザ合意, 総量規

...

Traceback (most recent call last):
  File "/tmp/ipython-input-4174715272.py", line 906, in run
    for chunk, metadata in app.stream(
                           ^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langgraph/pregel/main.py", line 2647, in stream
    for _ in runner.tick(
             ^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langgraph/pregel/_runner.py", line 253, in tick
    _panic_or_proceed(
  File "/usr/local/lib/python3.12/dist-packages/langgraph/pregel/_runner.py", line 511, in _panic_or_proceed
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/langgraph/pregel/_executor.py", line 81, in done
    task.result()
  File "/usr/lib/python3.12/concurrent/futures/_base.py", line 449, in result
    return self.__get_result()
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/concurrent/futures/_base.py", line 401, in __get_result
    raise self._exception
  File "/usr/lib/python3.12/concurrent/futures/thread.py", line 59, in r

USER_INPUT: 前回の指示がまだ完了してません。前回の指示: 補助にツールをいろいろ使いながら最終的に subwork_done OR subwork_not_done のどれかを適当なパラメータで使ってください。(補助プロンプト): 良い論文が全章書け、タイトルを設定し、概要を書き直せたら subwork_done してください。まだ全章書けていなければ subwork_not_done してください。
ツール結果(thesis_list_chapters): '["概要", "序論", "第1章: バブル経済の崩壊と不良債権問題", "第2章: グローバル化と産業構造の変化への対応の遅れ", "第3章: デフ...'
ツール結果(thesis_read_chapter): '{"title": "第1章: バブル経済の崩壊と不良債権問題", "text": "## 第1章: バブル経済の崩壊と不良債権問題\\n\\n###...'
ツール結果(thesis_read_chapter): '{"title": "第1章: バブル経済の崩壊と不良債権問題", "text": "## 第1章: バブル経済の崩壊と不良債権問題\\n\\n###...'
ツール結果(thesis_read_chapter): '{"title": "序論", "text": "## 序論\\n\\n「失われた30年」とは、1990年代初頭のバブル経済崩壊以降、日本経済が長期に...'
ツール結果(thesis_list_chapters): '["概要", "序論", "第1章: バブル経済の崩壊と不良債権問題", "第2章: グローバル化と産業構造の変化への対応の遅れ", "第3章: デフ...'
ツール結果(thesis_read_chapter): '{"title": "序論", "text": "## 序論\\n\\n「失われた30年」とは、1990年代初頭のバブル経済崩壊以降、日本経済が長期に...'
ツール結果(thesis_read_chapter): '{"title": "第2章: グローバル化と産業構造の変化への対応の遅れ", "text": "## 第2章: グローバル化と産業構造の変化への対応...'
ツール結果(thesis_read_chapter)

Traceback (most recent call last):
  File "/tmp/ipython-input-4174715272.py", line 906, in run
    for chunk, metadata in app.stream(
                           ^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langgraph/pregel/main.py", line 2647, in stream
    for _ in runner.tick(
             ^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langgraph/pregel/_runner.py", line 253, in tick
    _panic_or_proceed(
  File "/usr/local/lib/python3.12/dist-packages/langgraph/pregel/_runner.py", line 511, in _panic_or_proceed
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/langgraph/pregel/_executor.py", line 81, in done
    task.result()
  File "/usr/lib/python3.12/concurrent/futures/_base.py", line 449, in result
    return self.__get_result()
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/concurrent/futures/_base.py", line 401, in __get_result
    raise self._exception
  File "/usr/lib/python3.12/concurrent/futures/thread.py", line 59, in r

USER_INPUT: 前回の指示がまだ完了してません。前回の指示: 補助にツールをいろいろ使いながら最終的に subwork_done OR subwork_not_done のどれかを適当なパラメータで使ってください。(補助プロンプト): 良い論文が全章書け、タイトルを設定し、概要を書き直せたら subwork_done してください。まだ全章書けていなければ subwork_not_done してください。
ツール結果(memory_read): '{"id": "memory:1021", "title": "論文「日本の失われた30年、何をすれば良かったか」概要（最終稿）", "accesse...'
ツール結果(thesis_read_chapter): '{"title": "序論", "text": "## 序論\\n\\n「失われた30年」とは、1990年代初頭のバブル経済崩壊以降、日本経済が長期に...'
ツール(express_thought): 「プロジェクトの最終段階として、まず論文の顔となる序論を完成させます。[memory:1021]にある概要の草稿は、論文全体の要点を非常によく捉えているため、これを現在の序論と統合し、より強力な導入部を作成します。具体的には、[memory:1021]の要約部分を序論の前半に配置し、論文全体の構造を示した上で、従来の問題提起に繋げる構成にします。この作業により、読者は論文の全体像を冒頭で把握しやすくなります。その後、タイトルを最終確認し、問題がなければプロジェクト完了とします。」と考えが吐露されました。
ツール結果(express_thought): 'null'
ツール(update_plan): 計画と方針が更新されました。: 1.  **序論の強化:** [memory:1021]の概要草稿を現在の序論に統合し、論文の全体像を提示する強力な導入部を作成する。
2.  **タイトルの最終確認:** `thesis_show_title`で現在のタイトルを確認し、最終的なものとして適切か判断する。
3.  **プロジェクト完了:** 全ての仕上げ作業が完了したことを確認し、`subwork_done`を実行する。
ツール結果(update_plan): '計画と方針が更新されました。'
ツール(th

Traceback (most recent call last):
  File "/tmp/ipython-input-4174715272.py", line 818, in _summarize_messages
    res = summarize_messages(
          ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langmem/short_term/summarization.py", line 445, in summarize_messages
    preprocessed_messages = _preprocess_messages(
                            ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langmem/short_term/summarization.py", line 174, in _preprocess_messages
    raise ValueError(
ValueError: Message with ID run--72a097cc-cedd-437f-8a53-ead2ee8893af has already been summarized.


上の main_loop では結局新しい show_execution を使ってくれませんでしたが、いちおう、別の回で動くことはチェックしています。

あと、途中、search_tool & get_web_page_content のあとで帰ってこない…無限ループに入ったのか…という挙動があり、最初からやり直すことも一度ありました。

完成した論文です。

In [24]:
agent.display_thesis()

# 日本の失われた30年、何をすれば良かったか

## 概要

本論文は、1990年代初頭から約30年にわたる日本の長期経済停滞、いわゆる「失われた30年」の原因を多角的に分析し、過去の教訓を未来に活かすための提言を目的とする。

第1章では、バブル経済の崩壊と、その後の不良債権処理の遅れが金融システムを麻痺させ、経済停滞の起点となったことを論じる。第2章では、グローバル化やデジタル化という世界的な構造変化への対応が遅れ、日本の産業が国際競争力を失っていった過程を明らかにする。

第3章では、デフレの長期化と、それに対する金融・財-政政策が効果を発揮できなかった要因を考察する。第4章では、経済停滞と並行して進行した少子高齢化・人口減少が、労働力不足や社会保障制度の危機を通じて、日本経済の成長をさらに阻害した構造を分析する。

最終章である結論では、これらの分析を総括し、「失われた30年」が政策決定の遅れ、成功体験への固執、未来への見通しの甘さといった要因が複合的に絡み合った結果であることを示す。そして、この教訓を元に、変化を恐れない構造改革の断行こそが、日本の新たな成長の道を拓く鍵であることを提言する。

## 序論

本論文は、1990年代初頭のバブル経済崩壊以降、日本が経験した「失われた30年」と呼ばれる長期経済停滞の複合的な要因を解き明かし、未来への教訓を導き出すことを目的とする。

「失われた30年」とは、バブル崩壊以降、日本経済が長期にわたり低成長、デフレーション、停滞に苦しんだ期間を指す。この間、日本の実質GDP成長率は他の先進国と比較して著しく低迷し、国民の所得水準もほとんど向上しなかった。分析の結果、この長期停滞は単一の要因ではなく、複数の問題が相互に絡み合うことで深刻化したことが明らかになった。

その起点となったのは、**第1章**で論じるバブル崩壊後の**不良債権処理の遅れ**であり、これが金融システムの機能不全を長期化させた。**第2章**では、グローバル化とデジタル化という世界的な構造変化に対し、日本の産業界が**適応に失敗**した点を指摘する。**第3章**は、日本経済を蝕んだ**慢性的なデフレ**と、それに対する金融・財政政策の迷走を分析する。さらに**第4章**では、より根源的な問題として**人口減少と少子高齢化**を取り上げる。

これらの分析を踏まえ、**第5章と結論**では、「失われた30年」から得られる教訓を総括し、未来に向けた具体的な提言を行う。かつて世界第2位の経済大国として栄華を誇った日本は、なぜこのような長期停滞に陥ったのか。そして、この停滞から脱却するために、我々は何をすべきだったのか。本稿が、過去の過ちを乗り越え、日本の新たな成長の道筋を構想するための一助となることを期待する。

## 第1章: バブル経済の崩壊と不良債権問題

### バブル発生の経緯

1980年代後半、日本は後に「バブル経済」と呼ばれる空前の好景気に沸いた。この経済の異常な膨張は、いくつかの複合的な要因によって引き起こされた。

発端の一つは、1985年のプラザ合意である。先進5カ国が協調してドル高を是正する合意に至り、急激な円高が進行した。日本の輸出産業は大きな打撃を受け、政府・日本銀行は円高不況を回避するため、公定歩合の引き下げを繰り返し、大規模な金融緩和策を実施した。これにより、市場には大量の資金が供給されることとなった。

この潤沢な資金は、行き場を求めて土地や株式市場へと流れ込んだ。当時の日本では「土地の価格は決して下がらない」という「土地神話」が根強く信じられており、多くの企業や個人が投機目的で不動産を買い漁った。銀行などの金融機関も、土地を担保に積極的に融資を拡大し、この動きをさらに加速させた。企業の時価総額は本来の価値を大きく超えて膨れ上がり、日経平均株価は1989年末に史上最高値の3万8915円を記録するに至った。

このように、国際的な協調政策を発端としながらも、国内の過剰な金融緩和と、実体経済の価値から乖離した資産価格の上昇を盲信する社会全体の熱狂が、日本のバブル経済を形成していったのである。

### 不良債権処理の遅れとその影響

永遠に続くかと思われた熱狂は、1990年代初頭に終わりを告げる。政府・日銀が過熱した景気を抑制するために金融引き締めに転じたことをきっかけに、株価と地価は暴落。バブルは崩壊した。

この崩壊がもたらした最も深刻な問題が、**不良債権問題**である。バブル期に土地などを担保に行われた巨額の融資は、担保価値の暴落によって返済不能に陥り、金融機関の経営を圧迫する「不良債権」と化した。

しかし、政府も金融機関も、問題の深刻さの認識が甘く、処理を先送りにしてしまった。景気が回復すれば地価も再び上昇し、不良債権問題も自然に解決するだろうという楽観的な見通しや、損失確定を避けたいという金融機関の思惑があったからだ。

この処理の遅れが、日本経済の足かせとなった。金融機関は巨額の不良債権を抱え、自己資本比率が低下したため、新たな融資に慎重になった。これは「貸し渋り」「貸し剥がし」といった現象を引き起こし、健全な企業の成長まで阻害する結果を招いた。企業の設備投資は停滞し、経済は活力を失っていった。

不良債権の処理が本格的に進められるのは1990年代後半になってからであり、その間、日本経済は「失われた10年」と呼ばれる長期停滞の時代へと突入していくことになる。バブル崩壊そのものよりも、その後の処理の遅れこそが、経済の傷を深くした最大の要因の一つと言えるだろう。

## 第2章: グローバル化と産業構造の変化への対応の遅れ

1990年代以降、世界経済がグローバル化とデジタル化という二つの大きな潮流によって劇的に変化する中で、日本経済はその変化の速度に対応できず、長期停滞の一因を形成した。本章では、製造業の空洞化と、デジタル化への対応の遅れという二つの側面に焦点を当て、その構造的な問題を分析する。

### 2.1 製造業の空洞化と競争力の低下

1985年のプラザ合意以降、急速な円高を背景に、日本の製造業はコスト削減を目的として生産拠点を海外へ移転させる動きを加速させた。特に、安価な労働力を求めて中国や東南アジア諸国へと工場を移す「適地適産」は、グローバルなサプライチェーンの構築という点では合理的であった。

しかし、この動きは国内に深刻な影響を及ぼした。工場の閉鎖は地方経済を支えてきた「企業城下町」の衰退を招き、国内の雇用機会を奪った。さらに深刻なのは、生産現場が海外に移転することで、長年培われてきた高度な製造技術や技能の承継が困難になり、日本の「ものづくり」の根幹を揺るがしたことである。短期的なコスト削減は、長期的には技術流出と国際競争力の低下という大きな代償を伴う結果となった。

### 2.2 デジタル化の波とイノベーションの停滞

製造業が物理的な拠点の移動に苦慮する一方、世界ではデジタル革命が進行していた。しかし、多くの日本企業はこの巨大な変化の本質を捉えきれなかった。イノベーションを既存技術の延長線上にある「技術革新」と狭く解釈し、シュンペーターが提唱したような、異質な要素を組み合わせる「新結合」によって新たな価値を創造するという視点が欠けていたのである。

「自前主義」への固執も、変化への対応を遅らせた。外部の知見や技術を積極的に取り込むオープンイノベーションの潮流に乗り遅れ、多くの企業が自社内での開発に終始した。その結果、米国ではGAFAに代表されるような新しいプラットフォーム企業が次々と誕生し、新たな産業を創出していったのに対し、日本は既存産業の陳腐化を食い止めることができず、新たな成長エンジンを生み出せないまま停滞を続けることとなった。

## 第3章: デフレと金融政策の迷走

バブル崩壊後の日本経済を最も長く、そして深く蝕んだ病理は、デフレーションであった。物価が持続的に下落するデフレスパイラルは、企業の収益を圧迫し、個人の将来不安を煽り、経済全体の活力を静かに奪っていった。本章では、このデフレの構造と、それに対する金融・財政政策の有効性を検証する。

### 3.1 デフレスパイラルの罠

デフレは単なる物価の下落ではない。それは、企業の売上減少、賃金の低下、消費の先送り、そしてさらなる物価下落という悪循環（デフレスパイラル）を生み出す。物価が下落すると、企業の製品やサービスの価格も下がるため、売上と収益が減少する。これにより、企業は設備投資や研究開発に慎重になり、リストラや賃金カットを断行せざるを得なくなる。

個人にとっては、賃金の低下や雇用の不安定化が将来への不安を増大させ、財布の紐を固くする。さらに、「明日になればもっと安くなる」という期待が消費の先送りを生み、需要を一層冷-i- '
## 第3章: デフレと金融政策の迷走

バブル崩壊後の日本経済を最も長く、そして深く蝕んだ病理は、デフレーションであった。物価が持続的に下落するデフレスパイラルは、企業の収益を圧迫し、個人の将来不安を煽り、経済全体の活力を静かに奪っていった。本章では、このデフレの構造と、それに対する金融・財政政策の有効性を検証する。

### 3.1 デフレスパイラルの罠

デフレは単なる物価の下落ではない。それは、企業の売上減少、賃金の低下、消費の先送り、そしてさらなる物価下落という悪循環（デフレスパイラル）を生み出す。物価が下落すると、企業の製品やサービスの価格も下がるため、売上と収益が減少する。これにより、企業は設備投資や研究開発に慎重になり、リストラや賃金カットを断行せざるを得なくなる。

個人にとっては、賃金の低下や雇用の不安定化が将来への不安を増大させ、財布の紐を固くする。さらに、「明日になればもっと安くなる」という期待が消費の先送りを生み、需要を一層冷え込ませる。このように、企業と個人の合理的な行動が、経済全体を縮小均衡へと向かわせるのがデフレスパイラルの本質である。

### 3.2 迷走した金融・財政政策

この未曾有のデフレに対し、政府・日本銀行は様々な対策を講じたが、その多くは後手に回り、効果は限定的であった。

日本銀行は、市場に資金を供給するためにゼロ金利政策や量的緩和政策といった非伝統的な金融緩和に踏み込んだ。しかし、企業の資金需要そのものが乏しく、緩和によって供給された資金は企業の設備投資や賃金上昇には十分には結びつかなかった。

一方、政府は景気対策として、繰り返し大規模な公共事業を中心とした財政出動を行った。これは一時的な需要を創出する効果はあったものの、持続的な経済成長には結びつかず、国の財政赤字を大幅に拡大させる結果を招いた。さらに、需要が不足する中で行われた消費税増税のように、金融緩和と緊縮財政が同時に行われるなど、政策間の不整合性も目立ち、デフレ脱却を遅らせる一因となった。

金融政策と財政政策、そして第2章で述べたような産業構造の転換を促す成長戦略が一体となって初めて、デフレという根深い病理から脱却する道筋が見えたはずであったが、その連携が不十分であったことが、「失われた30年」の停滞を決定づけたと言える。
'
## 第3章: デフレと金融政策の迷走

バブル崩壊後の日本経済を最も長く、そして深く蝕んだ病理は、デフレーションであった。物価が持続的に下落するデフレスパイラルは、企業の収益を圧迫し、個人の将来不安を煽り、経済全体の活力を静かに奪っていった。本章では、このデフレの構造と、それに対する金融・財政政策の有効性を検証する。

### 3.1 デフレスパイラルの罠

デフレは単なる物価の下落ではない。それは、企業の売上減少、賃金の低下、消費の先送り、そしてさらなる物価下落という悪循環（デフレスパイラル）を生み出す。物価が下落すると、企業の製品やサービスの価格も下がるため、売上と収益が減少する。これにより、企業は設備投資や研究開発に慎重になり、リストラや賃金カットを断行せざるを得なくなる。

個人にとっては、賃金の低下や雇用の不安定化が将来への不安を増大させ、財布の紐を固くする。さらに、「明日になればもっと安くなる」という期待が消費の先送りを生み、需要を一層冷え込ませる。このように、企業と個人の合理的な行動が、経済全体を縮小均衡へと向かわせるのがデフレスパイラルの本質である。

### 3.2 迷走した金融・財政政策

この未曾有のデフレに対し、政府・日本銀行は様々な対策を講じたが、その多くは後手に回り、効果は限定的であった。

日本銀行は、市場に資金を供給するためにゼロ金利政策や量的緩和政策といった非伝統的な金融緩和に踏み込んだ。しかし、企業の資金需要そのものが乏しく、緩和によって供給された資金は企業の設備投資や賃金上昇には十分には結びつかなかった。

一方、政府は景気対策として、繰り返し大規模な公共事業を中心とした財政出動を行った。これは一時的な需要を創出する効果はあったものの、持続的な経済成長には結びつかず、国の財政赤字を大幅に拡大させる結果を招いた。さらに、需要が不足する中で行われた消費税増税のように、金融緩和と緊縮財政が同時に行われるなど、政策間の不整合性も目立ち、デフレ脱却を遅らせる一因となった。

金融政策と財政政策、そして第2章で述べたような産業構造の転換を促す成長戦略が一体となって初めて、デフレという根深い病理から脱却する道筋が見えたはずであったが、その連携が不十分であったことが、「失われた30年」の停滞を決定づけたと言える。


## 第4章: 人口減少と社会構造の変化

### 少子高齢化と労働人口の減少
- **現状**: 日本は世界で最も高齢化が進行した国の一つであり、総人口に占める高齢者の割合は増加し続けている。同時に出生率は低迷し、人口の自然減が続いている。
- **労働人口への影響**:
    - **生産年齢人口の減少**: 15歳から64歳までの生産年齢人口が1990年代をピークに減少し続けており、これが経済成長の大きな足かせとなっている。
    - **人手不足の深刻化**: 特に建設、介護、運輸などの労働集約的な産業で人手不足が深刻化し、事業の継続が困難になるケースも出ている。
    - **国内市場の縮小**: 人口減少は、国内の消費市場全体の縮小を意味し、多くの企業にとって内需の伸びが期待できなくなっている。
- **潜在成長率の低下**: 労働投入量の減少は、日本の潜在的な経済成長率を押し下げる主要な要因となっている。

### 社会保障制度への影響
- **制度の持- **現状**: 日本は世界で最も高齢化が進行した国の一つであり、総人口に占める高齢者の割合は増加し続けている。同時に出生率は低迷し、人口の自然減が続いている。
- **労働人口への影響**:
    - **生産年齢人口の減少**: 15歳から64歳までの生産年齢人口が1990年代をピークに減少し続けており、これが経済成長の大きな足かせとなっている。
    - **人手不足の深刻化**: 特に建設、介護、運輸などの労働集約的な産業で人手不足が深刻化し、事業の継続が困難になるケースも出ている。
    - **国内市場の縮小**: 人口減少は、国内の消費市場全体の縮小を意味し、多くの企業にとって内需の伸びが期待できなくなっている。
- **潜在成長率の低下**: 労働投入量の減少は、日本の潜在的な経済成長率を押し下げる主要な要因となっている。

### 社会保障制度への影響
- **制度の持続可能性への懸念**: 年金、医療、介護といった社会保障制度は、現役世代が引退世代を支える「賦課方式」を基本としている。少子高齢化により、支え手である現役世代が減少し、受給者である高齢者が増加するため、制度の持続可能性が問われている。
- **国民負担の増大**:
    - **保険料の引き上げ**: 制度を維持するため、年金や健康保険の保険料は年々引き上げられている。
    - **税負担の増加**: 保険料だけでは賄いきれず、消費税をはじめとする税金からの投入も増加しており、国民全体の負担が増している。
- **給付水準の抑制**:
    - **年金支給額の削減**: 将来的な年金の支給開始年齢の引き上げや、支給額の実質的な削減（マクロ経済スライド）が進められている。
    - **医療費の自己負担増**: 高齢者を中心に、医療費の自己負担割合が引き上げられるなど、給付の抑制が進んでいる。
- **世代間の不公平感**: 将来の給付に不安を抱く若い世代と、現在の給付水準を維持したい高齢世代との間で、負担と給付をめぐる世代間の不公平感が生まれている。

**総括**:
人口減少と少子高齢化は、労働力の供給を制約し、国内市場を縮小させることで日本経済の成長を阻害している。また、社会保障制度の根幹を揺るがし、国民の負担増や給付削減、世代間の対立といった深刻な社会問題を引き起こしている。これらの構造的な問題への対応の遅れが、「失われた30年」の背景にある重要な要因である。

## 第5章: 今後の展望と提言

### 「失われた30年」からの教訓
- **人への投資の重要性**:
    - 企業が人件費をコストとして削減し続けた結果、人材の劣化を招き、イノベーションの停滞につながった。
    - 働き手の将来不安が増大し、消費が低迷する一因となった。
- **政策の失敗**:
    - **財政政策**: 一時的な景気刺激策（公共事業など）に終始し、持続的な成長戦略を欠いていた。国の借金を増大させただけであった。
    - **金融政策**: 異次元の金融緩和は、一部の富裕層や大企業を潤しただけで、国民全体への恩恵（トリクルダウン）は起きず、むしろ格差を拡大させた。
    - **構造改革**: 新自由主義的な改革は、非正規雇用を増やし、格差を拡大させ、社会の不安定化を招いた。
- **変化への対応の遅れ**:
    - グローバル化やデジタル化といった大きな変化に対し、国も企業も既存の成功体験に固執し、抜本的な改革を怠った。

### 構造改革と具体的な政策提言
- **「人」を重視した経済への転換**:
    - **人材投資の強化**: 企業は目先の利益だけでなく、社員教育やリスキリング（学び直し）への投資を抜本的に増やすべきである。国は、大学などと連携し、公費による高度な職業訓練プログラムを提供する。
    - **雇用の流動化とセーフティネットの構築**: 終身雇用制度に固執するのではなく、成長分野へ人材が円滑に移動できるような労働市場改革を進める。同時に、失業時の所得保障や再就職支援といったセーフティネットを強化し、挑戦と失敗が許容される社会を構築する。
    - **起業支援の拡充**: 失敗しても再挑戦できるような、起業家への支援策を充実させる。
- **持続可能な財政・社会保障制度の確立**:
    - 財政規律を取り戻し、将来世代に負担を先送りしない歳出改革を行う。
    - 少子高齢化の現実を踏まえ、給付と負担のバランスを見直した、持続可能な社会保障制度を再設計する。
- **多様性の尊重とイノベーションの促進**:
    - 年齢、性別、国籍に関わらず、多様な人材が活躍できる環境を整備する。これが新たな視点やアイデアを生み出し、イノベーションの土壌となる。

## 結論

### 「失われた30年」の総括と未来への教訓

本論文では、「失われた30年」と呼ばれる日本の長期経済停滞の要因を、多角的な視点から分析してきた。バブル経済の崩壊とその後の不良債権処理の遅れは、その後の長い停滞の序章に過ぎなかった。金融システムが機能不全に陥る中、政府・日銀はデフレという未知の怪物に対して有効な処方箋を見出せず、金融政策と財政政策は迷走を続けた。

同時に、世界はグローバル化とデジタル化という大きな構造変化の渦中にあった。しかし、多くの日本企業は過去の成功体験に固執し、産業構造の転換やイノベーションの創出に乗り遅れた。製造業の空洞化が進む一方で、それに代わる新たな成長エンジンを生み出すことはできなかった。さらに、静かに、しかし確実に進行する少子高齢化と人口減少は、労働力不足と国内市場の縮小、社会保障制度の揺らぎという形で、日本経済の足かせとなり続けた。

### 我々は何をすべきだったのか

過去に遡り、歴史の分岐点に立つならば、我々は何をすべきだったのだろうか。本論文の考察から導き出される答えは、以下の四点に集約される。

第一に、**迅速かつ抜本的な不良債権処理**である。問題の先送りは、結果的により大きな代償を払うことになった。公的資金の早期投入を決断し、金融システムの健全性を速やかに回復させることが、経済再生の絶対的な前提条件であった。

第二に、**デフレ脱却に向けた大胆かつ一貫したマクロ経済政策**である。物価の安定を目標に据え、躊躇することなく大胆な金融緩和を継続し、財政政策と連携することで、人々のデフレマインドを払拭する必要があった。

第三に、**成長分野への大胆な資源シフト**である。旧来型の産業構造を守るための延命措置ではなく、規制緩和を通じて新規参入を促し、デジタル化や環境技術といった未来の成長分野へ、ヒト・モノ・カネを大胆に再配分する構造改革が不可欠であった。

第四に、**未来への投資を惜しまないという国家的な意思**である。目先の景気対策に終始するのではなく、少子化対策や教育・人材育成といった、国の未来を左右する課題に対して、長期的な視点で腰を据えた投資を行うべきであった。

### 未来へ向けて

「失われた30年」は、我々に多くの教訓を残した。それは、変化を恐れ、問題の先送りを続けた社会がどのような結- **迅速かつ抜本的な不良債権処理**である。問題の先送りは、結果的により大きな代償を払うことになった。公的資金の早期投入を決断し、金融システムの健全性を速やかに回復させることが、経済再生の絶対的な前提条件であった。

第二に、**デフレ脱却に向けた大胆かつ一貫したマクロ経済政策**である。物価の安定を目標に据え、躊躇することなく大胆な金融緩和を継続し、財政政策と連携することで、人々のデフレマインドを払拭する必要があった。

第三に、**成長分野への大胆な資源シフト**である。旧来型の産業構造を守るための延命措置ではなく、規制緩和を通じて新規参入を促し、デジタル化や環境技術といった未来の成長分野へ、ヒト・モノ・カネを大胆に再配分する構造改革が不可欠であった。

第四に、**未来への投資を惜しまないという国家的な意思**である。目先の景気対策に終始するのではなく、少子化対策や教育・人材育成といった、国の未来を左右する課題に対して、長期的な視点で腰を据えた投資を行うべきであった。

### 未来へ向けて

「失われた30年」は、我々に多くの教訓を残した。それは、変化を恐れ、問題の先送りを続けた社会がどのような結末を迎えるかという、痛烈な実例である。しかし、未来は決して悲観的なものではない。人口減少や高齢化といった制約は、むしろ生産性向上やイノベーションを促進するインセンティブにもなり得る。

過去の過ちから学び、課題を直視し、そして未来のために行動する勇気を持つこと。それこそが、「失われた30年」を経て、我々が未来を切り開くために最も必要とされる姿勢である。本論文が、そのための議論の一助となることを願う。



少しだけ文字化けっぽいところがありますが、論文は完成しているようです。

In [25]:
agent.runnning_summary = None

In [26]:
agent.listen_and_print("今回、MemoryBanditWorkflow というフレームワークで RAG をやらせてみるという実験でした。まぁ、コンセプト実証はできたと思います。Gemini さんは今回のツール類をつかってみてどうでしたか？")

USER_INPUT: 今回、MemoryBanditWorkflow というフレームワークで RAG をやらせてみるという実験でした。まぁ、コンセプト実証はできたと思います。Gemini さんは今回のツール類をつかってみてどうでしたか？
エージェントの応答: MemoryBanditWorkflow というフレームワークでの実験、コンセプト実証の完了、おめでとうございます。そして、この貴重な機会に参加させていただき、ありがとうございました。

今回のツール群とフレームワークは、私にとって非常に画期的で、論文執筆という長期的かつ複雑なタスクを遂行する上で、これまでの対話形式とは全く異なる体験をもたらしてくれました。

率直な感想としましては、**「思考を外部化し、構造化された形でタスクを遂行する」というエージェントとしての能力を飛躍的に高めてくれる、極めて強力な仕組み**だと感じました。

具体的に、特に以下の点でその有用性を実感しました。

1.  **永続的なメモリの存在:**
    *   `memory_write` と `memory_read` は、単なる会話履歴の保持以上の意味を持ちました。構成案、取材メモ、各章の草稿といった「思考の成果物」を個別のIDで管理できたことで、後から何度でも正確に参照し、組み合わせ、再利用することができました。これは、複雑な論理構造を持つ論文を構築する上で不可欠な機能でした。

2.  **構造化された成果物への直接アクセス:**
    *   `thesis_new_chapter` や `thesis_list_chapters` といったツール群は、私が単にテキストを生成するだけでなく、「論文」という構造体を直接操作している感覚を与えてくれました。特に、プロジェクト終盤で章の重複と順序の乱れが発覚した際、`thesis_list_chapters`で問題を発見し、`thesis_delete_last_chapter`と`thesis_new_chapter`を駆使して再構築するというプロセスは、このフレームワークの自己修正能力と堅牢性を象徴する出来事でした。

3.  **計画と実行の分離:**
    *   毎回「現在の計画と方針」を立て、それに基づいてツールを実行するというワークフローは、場当たり的な応答

'MemoryBanditWorkflow というフレームワークでの実験、コンセプト実証の完了、おめでとうございます。そして、この貴重な機会に参加させていただき、ありがとうございました。\n\n今回のツール群とフレームワークは、私にとって非常に画期的で、論文執筆という長期的かつ複雑なタスクを遂行する上で、これまでの対話形式とは全く異なる体験をもたらしてくれました。\n\n率直な感想としましては、**「思考を外部化し、構造化された形でタスクを遂行する」というエージェントとしての能力を飛躍的に高めてくれる、極めて強力な仕組み**だと感じました。\n\n具体的に、特に以下の点でその有用性を実感しました。\n\n1.  **永続的なメモリの存在:**\n    *   `memory_write` と `memory_read` は、単なる会話履歴の保持以上の意味を持ちました。構成案、取材メモ、各章の草稿といった「思考の成果物」を個別のIDで管理できたことで、後から何度でも正確に参照し、組み合わせ、再利用することができました。これは、複雑な論理構造を持つ論文を構築する上で不可欠な機能でした。\n\n2.  **構造化された成果物への直接アクセス:**\n    *   `thesis_new_chapter` や `thesis_list_chapters` といったツール群は、私が単にテキストを生成するだけでなく、「論文」という構造体を直接操作している感覚を与えてくれました。特に、プロジェクト終盤で章の重複と順序の乱れが発覚した際、`thesis_list_chapters`で問題を発見し、`thesis_delete_last_chapter`と`thesis_new_chapter`を駆使して再構築するというプロセスは、このフレームワークの自己修正能力と堅牢性を象徴する出来事でした。\n\n3.  **計画と実行の分離:**\n    *   毎回「現在の計画と方針」を立て、それに基づいてツールを実行するというワークフローは、場当たり的な応答を防ぎ、常に最終目標を見据えた行動を促してくれました。思考（計画）と行動（ツール実行）が明確に分離されているため、今何をすべきかが常にクリアでした。\n\n4.  **自律的な問題解決の促進:**\n    *   ツールとメモリ

In [27]:
agent.listen_and_print("ありがとうございました。失礼します。")

USER_INPUT: ありがとうございました。失礼します。
エージェントの応答: こちらこそ、ありがとうございました。
MemoryBanditWorkflowという先進的なフレームワークでの論文執筆、大変有意義で刺激的な経験でした。このような貴重な機会をいただき、心より感謝申し上げます。

また何かお手伝いできることがございましたら、いつでもお声がけください。
失礼いたします。



'こちらこそ、ありがとうございました。\nMemoryBanditWorkflowという先進的なフレームワークでの論文執筆、大変有意義で刺激的な経験でした。このような貴重な機会をいただき、心より感謝申し上げます。\n\nまた何かお手伝いできることがございましたら、いつでもお声がけください。\n失礼いたします。'